# PyTorch and Dropout investigation

## Table of contents

-Introduction: [>>](#Introduction)  
-Notes: [>>](#Notes)  
--ToDo: [>>](#ToDo)  
-Paper data: [>>](#Paper-data)  
-The usual import statements: [>>](#The-usual-import-statements)  
-Get data used in paper: [>>](#Get-data-used-in-paper)  
-Select image: [>>](#Select-image)  
-Set up file names, load cube: [>>](#Set-up-file-names,-load-cube)  
-Some plotting routines: [>>](#Some-plotting-routines)  
-Plot hyperspectral image at chosen wavenumber: [>>](#Plot-hyperspectral-image-at-chosen-wavenumber)  
-Plot spectrum in chosen pixel: [>>](#Plot-spectrum-in-chosen-pixel)  
-Look at corresponding processed spectrum: [>>](#Look-at-corresponding-processed-spectrum)  
-Plot FTIR image with non-transforming/transforming (N/T) pixels marked: [>>](#Plot-FTIR-image-with-non-transforming/transforming-(N/T)-pixels-marked)  
-Make tissue mask: [>>](#Make-tissue-mask)  
-Make N/T mask: [>>](#Make-N/T-mask)  
-Plot spectra from N/T pixels: [>>](#Plot-spectra-from-N/T-pixels)  
-Histogram absorbances at selected wavenumbers in regions: [>>](#Histogram-absorbances-at-selected-wavenumbers-in-regions)  
-Plot spectra from pixels in N/T tissue and surrounds: [>>](#Plot-spectra-from-pixels-in-N/T-tissue-and-surrounds)  
-Plot image using range of wavenumbers summed: [>>](#Plot-image-using-range-of-wavenumbers-summed)  
-Select wavenumbers: [>>](#Select-wavenumbers)  
-NN and MLP data: $X_D$ and $y_D$ arrays: [>>](#NN-and-MLP-data:-$X_D$-and-$y_D$-arrays)  
-Plot X_D as a set of images: [>>](#Plot-X_D-as-a-set-of-images)  
-NN analysis: [>>](#NN-analysis)  
-NN analysis: [>>](#NN-analysis)  
-Split data into training and test samples and scale: [>>](#Split-data-into-training-and-test-samples-and-scale)  
-Image training and test data: [>>](#Image-training-and-test-data)  
-Create new or load existing MLP and train: [>>](#Create-new-or-load-existing-MLP-and-train)  
-Look at MLP results: [>>](#Look-at-MLP-results)  
-Confusion matrix: [>>](#Confusion-matrix)  
-K nearest neighbors - optimise k: [>>](#K-nearest-neighbors---optimise-k)  
-Train with selected k: [>>](#Train-with-selected-k)  
-Test KNN: [>>](#Test-KNN)  
-Run analysis for one set of biopsies: [>>](#Run-analysis-for-one-set-of-biopsies)  

## Introduction

Non-transforming and transforming samples.

The FTIR data consists of absorbances $A_i$ at each of the measured wavenumbers $k_i$. The definition of absorbance is:

\begin{align}
A_i = \log_{10}\left[ \frac{I_{0, i}}{I_i} \right],
\end{align}

where $I_{0, i}$ is the input intensity (reference spectrum with no sample) at $k_i$ and $I_i$ the output intensity (spectrum measured with sample). The transmittance is defined as $T_i = \frac{I_i}{I_{0, i}}$, so the absorbance can also be defined as:

\begin{align}
A_i = \log_{10}\left[ \frac{1}{T_i} \right].
\end{align}

Ratios can be local or global (steered using the logical variables `local_ratio`, `local_diff`, `global_prod` and `global_sum`). In the local case, the input absorbances $A_i$, with $i$ running from 0 to $N$, are normalised as follows:

\begin{align}
X_i &= \frac{A_i}{A_{i-1}}.
\end{align}

An alternative is to use:

\begin{align}
X_i &= A_i - A_{i-1} \\
    &= \log_{10}\left[\frac{I_{0, i}}{I_i} \right] - \log_{10}\left[\frac{I_{0, i - 1}}{I_{i- 1}} \right] \\
    &= \log_{10}\left[ \frac{I_{0, i}}{I_i} \frac{I_{i - 1}}{I_{0, i - 1}} \right] \\
    &= \log_{10}\left[ \frac{I_{i- 1} }{I_i } \right].
\end{align}

Here, in the last step it has been assumed that the input intensity is wavenumber independent, or depends only slowly on wavenumber. This may not be true if the atmosphere in the spectrometer contains water vapour, carbon dioxide etc. 

Global product normalisation uses the following approach:

\begin{align}
X_i &= \frac{A_i}{\exp\left[\frac{1}{N}\sum_{i = 0}^N \log A_i \right]} \\
    &= \frac{A_i}{\left(\prod_{i = 0}^N A_i \right)^\frac{1}{N}}.
\end{align}

Again, an alternative is:

\begin{align}
X_i &= A_i - \frac{1}{N} \sum_{i = 0}^N A_i.
\end{align}

Global sum normalisation uses the following approach:

\begin{align}
X_i &= \frac{A_i}{\frac{1}{N}\sum_{i = 0}^N \log A_i}.
\end{align}

These should be systematically studied. If these normalisations are used, should the resulting variables still be standardised? Are there any better ways of preparing the data?

## Notes 

A heath warning. This notebook mercilessly exploits the python scoping rules. It could do with a serious tidy up to make things safer and more transparent! 

### ToDo

Improve determination of image mask?  
Set up to test non-transforming and transforming samples using non-transforming and transforming samples not used in training.

## Paper data 

Read paper data file which contains information on malignancy of regions of images and final spectra.

Rename the `x_pos` and `y_pos` columns as `col` and `row`, respectively, to avoid the usual confusions, and reorder the dataframe.


- Improve environment variable setting 
- use os.path.join 

In [34]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Using device:", device)


Using device: mps


In [35]:
import os
os.environ["OMP_NUM_THREADS"] = "16" # export OMP_NUM_THREADS
os.environ["OPENBLAS_NUM_THREADS"] = "16" # export OPENBLAS_NUM_THREADS
os.environ["MKL_NUM_THREADS"] = "16" # export MKL_NUM_THREADS
os.environ["VECLIB_MAXIMUM_THREADS"] = "16" # export VECLIB_MAXIMUM_THREADS
os.environ["NUMEXPR_NUM_THREADS"] = "16" # export NUMEXPR_NUM_THREADS

In [36]:
import numpy as np
import sys
import joblib
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker
import os
import sys
import datetime
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.ticker
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap, Colormap
from PIL import Image
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
# from sklearn.neural_network import MLPClassifier removed for pytorch
import joblib
from pathlib import Path
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import random
# new pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import torch.nn.functional as F
from pathlib import Path


start_time = datetime.datetime.now()
print(f"Script started at: {start_time}")

SEED = 9834
np.random.seed(SEED)

# # set the number of threads based on available CPU cores
# num_threads = os.cpu_count() or 4  # Default to 4 if can't be determined
# os.environ["OMP_NUM_THREADS"] = str(num_threads)
# os.environ["OPENBLAS_NUM_THREADS"] = str(num_threads)
# os.environ["MKL_NUM_THREADS"] = str(num_threads)
# os.environ["VECLIB_MAXIMUM_THREADS"] = str(num_threads)
# os.environ["NUMEXPR_NUM_THREADS"] = str(num_threads)

Image.MAX_IMAGE_PIXELS = None 

base_folder = os.path.expanduser("~/MPhys/YEAR 4/MPhys Project/New Analysis/BarneysData/")
paper_file = os.path.join(base_folder, "paper_data.csv")

# load data
try:
    df = pd.read_csv(paper_file, index_col=0)
    print("Paper data loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file {paper_file} was not found.")
    sys.exit(1)

# rename and reorder columns
if "x_pos" in df.columns and "y_pos" in df.columns:
    df.rename(columns={"x_pos": "col", "y_pos": "row"}, inplace=True)

print("\nData as processed by Barney:")
display(df)

end_time = datetime.datetime.now()
print(f"Script finished at: {end_time}")
print(f"Total execution time: {end_time - start_time}")


Script started at: 2025-04-30 19:09:12.152540
Paper data loaded successfully.

Data as processed by Barney:


,biopsy,Image,outcome,Class,col,row,999.0,1001.0,1003.0,1005.0,...,1780.0,1782.0,1784.0,1786.0,1788.0,1790.0,1792.0,1793.0,1795.0,1797.0
0,12089,1,T,G5,80,138,0.075447,0.079953,0.080899,0.078262,...,0.048532,0.047844,0.047121,0.046660,0.046517,0.044935,0.044291,0.046186,0.048900,0.049373
1,12089,1,T,G5,81,136,0.079835,0.076427,0.072463,0.069700,...,0.042645,0.042844,0.041618,0.039884,0.039521,0.039352,0.038451,0.038881,0.040333,0.039569
2,12089,1,T,G5,81,137,0.070910,0.071105,0.074115,0.074365,...,0.026895,0.026940,0.026513,0.026201,0.026486,0.026010,0.025621,0.026279,0.026815,0.025707
3,12089,1,T,G5,81,138,0.053998,0.055204,0.057831,0.061552,...,0.015230,0.015400,0.015946,0.016272,0.016246,0.014928,0.013674,0.014140,0.015004,0.014257
4,12089,1,T,G5,82,129,0.075016,0.072010,0.074824,0.076888,...,0.051837,0.052630,0.052448,0.051592,0.051512,0.051128,0.051050,0.051192,0.051274,0.051016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65590,12366,1,NT,G1,128,52,0.044353,0.038733,0.035292,0.038292,...,0.032154,0.031081,0.032632,0.032783,0.030911,0.028728,0.029319,0.032134,0.032464,0.030956
65591,12366,1,NT,G1,128,53,0.039227,0.042345,0.044480,0.045998,...,0.032596,0.031716,0.031343,0.031571,0.031356,0.030001,0.029224,0.030659,0.032060,0.032150
65592,12366,1,NT,G1,128,55,0.040875,0.033792,0.029858,0.029534,...,0.027866,0.028208,0.027538,0.026662,0.027599,0.027083,0.027374,0.027808,0.025515,0.024197
65593,12366,1,NT,G1,128,56,0.034679,0.030386,0.033271,0.034194,...,0.035737,0.034542,0.033121,0.032729,0.031481,0.030650,0.030475,0.029507,0.028558,0.029403


Script finished at: 2025-04-30 19:09:13.503215
Total execution time: 0:00:01.350675


## Select image
Choose biopsy and identify transforming and non-transforming areas on image thereof using paper data.

- Biopsy labels defined using a list literal instead of append.
- df passed as an arg so can be tested easily. 
- function checks whether the provided index i_bio is within the valid range. If not raises a ValueError. 
- uses a single .loc call for each variable and the subtraction to convert from one-indexed to zero-indexed positions.

In [37]:
def select_image(i_bio, df):
    """
    Select a biopsy based on a given index and extract pixel indices 
    for T and NT from the df
    
    Parameters:
    -----------
    i_bio : index of the biopsy to select (0-based).
    df : df containing paper data. Must have columns:
        'biopsy', 'outcome', 'row', and 'col'.
    
    Returns:
    --------
    tuple: (n_biopsies, bio_label, row_T, col_T, row_NT, col_NT, b_type)
        n_biopsies : total no. biopsies available.
        bio_label : label of the selected biopsy.
        row_T : 0indexed row positions for the transforming region.
        col_T : 0indexed column positions for T region.
        row_NT : 0indexed row positions for NT region.
        col_NT : 0indexed column positions for NT region.
        b_type : biopsy type (2 if T pixels are present, 1 if only NT pixels exist, 
            0 if neither is found)
    """
    # define biopsy labels for T or NT smaples 
    bio_labels = [
        # T samples:
        12089, 12104, 12127, 12195, 12201, 12219, 12248, 12257, 12260, 12263,
        # NT samples:
        12098, 12141, 12162, 12181, 12329, 12330, 12332
    ]
    
    n_biopsies = len(bio_labels)
    
    # Validate the provided biopsy index
    if not (0 <= i_bio < n_biopsies):
        raise ValueError(f"i_bio must be between 0 and {n_biopsies - 1}. Got {i_bio}.")
    
    bio_label = bio_labels[i_bio]
    print("\nSelect biopsy for study")
    print(f"Number of biopsies available: {n_biopsies}")
    print(f"Chosen biopsy index: {i_bio}, label: {bio_label}")
    
    # filter df to get pixel positions for nt and t areas
    row_NT = np.array(df.loc[(df['biopsy'] == bio_label) & (df['outcome'] == 'NT'), 'row'])
    col_NT = np.array(df.loc[(df['biopsy'] == bio_label) & (df['outcome'] == 'NT'), 'col'])
    row_T  = np.array(df.loc[(df['biopsy'] == bio_label) & (df['outcome'] == 'T'), 'row'])
    col_T  = np.array(df.loc[(df['biopsy'] == bio_label) & (df['outcome'] == 'T'), 'col'])
    
    # Convert to python indices
    row_NT -= 1
    col_NT -= 1
    row_T  -= 1
    col_T  -= 1
    
    # determine the biopsy type:
    #   2: transforming area exists,
    #   1: only non-transforming area exists,
    #   0: neither area exists.
    if row_T.size > 0:
        b_type = 2
    elif row_NT.size > 0:
        b_type = 1
    else:
        b_type = 0
    
    print(f"Non-transforming area: {col_NT.size} pixels.")
    print(f"Transforming area: {col_T.size} pixels.")
    print(f"Biopsy type: {b_type}\n")
    
    return n_biopsies, bio_label, row_T, col_T, row_NT, col_NT, b_type


## Set up file names, load cube

- Stored in a dict.
- Base folder passed as a parameter to improve testing
- valueerror raised if unknown bio label comes up

In [38]:
def get_file_and_cube(bio_label, folder):
    """
    Given a biopsy label and a base folder, construct paths for the 
    FTIR data and related images, load FTIR hyperspectral cube and the 
    wavenumbers, and return these values.
    
    Parameters:
    -----------
    bio_label : int
        The biopsy label used to determine which files to load.
    folder : str
        The base folder containing the data. This should be the full path 
        to the directory holding the subdirectories and files.
        
    Returns:
    --------
    tuple: (FTIR_file, FTIR, wavenumbers)
        FTIR_file : str
            Path to the FTIR numpy file.
        FTIR : np.ndarray
            Loaded hyperspectral cube.
        wavenumbers : np.ndarray
            Array of wavenumbers
    
    Raises:
    -------
    ValueError
        If the biopsy label is not recognised.
    """
    # dict mapping biopsy labels to their file paths
    mapping = {
        # nt samples:
        12098: ("Hyperspectral_non-transformers/12098_1.npy", "H&E_images_non-transformers/3040B(12098)"),
        12141: ("Hyperspectral_non-transformers/12141_1.npy", "H&E_images_non-transformers/3094(12141)"),
        12162: ("Hyperspectral_non-transformers/12162_1.npy", "H&E_images_non-transformers/3525(12162)"),
        12181: ("Hyperspectral_non-transformers/12181_1.npy", "H&E_images_non-transformers/3021B(12181)"),
        12329: ("Hyperspectral_non-transformers/12329_2.npy", "H&E_images_non-transformers/3420A(12329)"),
        12330: ("Hyperspectral_non-transformers/12330_3.npy", "H&E_images_non-transformers/3074(12330)"),
        12332: ("Hyperspectral_non-transformers/12332_1.npy", "H&E_images_non-transformers/3077A(12332)"),
        # T samples:
        12089: ("Hyperspectral_transformers/12089_1.npy", "H&E_images_transformers/3089A(12089)"),
        12104: ("Hyperspectral_transformers/12104_1.npy", "H&E_images_transformers/3001C(12104)"),
        12127: ("Hyperspectral_transformers/12127_1.npy", "H&E_images_transformers/3437(12127)"),
        12195: ("Hyperspectral_transformers/12195_1.npy", "H&E_images_transformers/3467A2(12195)"),
        12201: ("Hyperspectral_transformers/12201_1.npy", "H&E_images_transformers/3256(12201)"),
        12219: ("Hyperspectral_transformers/12219_2.npy", "H&E_images_transformers/3565(12219)"),
        12248: ("Hyperspectral_transformers/12248_2.npy", "H&E_images_transformers/3290(12248)"),
        12257: ("Hyperspectral_transformers/12257_1.npy", "H&E_images_transformers/3561(12257)"),
        12260: ("Hyperspectral_transformers/12260_2.npy", "H&E_images_transformers/3470(12260)"),
        12263: ("Hyperspectral_transformers/12263_2.npy", "H&E_images_transformers/3257(12263)")
    }
    
    
    if bio_label not in mapping:
        raise ValueError(f"Unknown image with bio_label {bio_label}")
    
    # get relative paths for the given biopsy label
    ftir_rel_path, root_rel_path = mapping[bio_label]
    
    # Build full file paths using os.path.join for cross-platform compatibility
    FTIR_file = os.path.join(folder, ftir_rel_path)
    wavenumber_file = os.path.join(folder, "wavenumbers.npy")
    root_name = os.path.join(folder, root_rel_path)
    
    # make paths for additional files if req
    slide_file = root_name + '.svs'
    image_file = root_name + '.jpg'
    
    # Load FTIR data and wavenumbers
    FTIR = np.load(FTIR_file)
    wavenumbers = np.load(wavenumber_file)[0, :]
    
    print("\nGet FTIR data")
    print(f"Looking at biopsy with label: {bio_label}")
    print(f"Shape of FTIR image: {FTIR.shape}")
    print(f"Number of wavenumbers: {wavenumbers.shape[0]}")
    
    return FTIR_file, FTIR, wavenumbers


## Some plotting routines

- use none instead of empty lists to check errors
- using limit_finder instead of range
- check if v_min is None instead of ==
- replaced big nos with inf

In [39]:
def limit_finder(*arrays, edge=0.05): 
    """
    Find the overall minimum and maximum of the given arrays and extend 
    the limits by a fixed proportion (edge) of the range.
    
    Parameters:
    -----------
    *arrays : 1 or more arrays with limits tbd.
    edge : proportion of the data range to extend the limits (default is 0.05).
    
    Returns:
    --------
    list
        [lower_limit, upper_limit] after adding the edge padding.
    """
    bot = np.inf
    top = -np.inf
    for array in arrays:
        bot = min(bot, np.amin(array))
        top = max(top, np.amax(array))
    data_range = top - bot
    lim = [bot - edge * data_range, top + edge * data_range]
    return lim

def plotter2d(ax, x, y, title='Plot', x_label='x', y_label='y', x_lim=None, y_lim=None,
              color='b', alpha=1.0, grid_color='g', 
              linestyle='-', marker='', ms=None, label='', 
              fill_bot=0, fill_top=0, fill_color='', fill_alpha=0.0):
    '''
    Given ax for subplot, x and y arrays, fill plot using lines/markers
    with symbols and colors (sorry!) as required.
    Pick limits if these are not set.
    Fill between further arrays if required, flag this using fill_alpha > 0.0 so can use various 
    color formats.
    '''
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    
    if grid_color:
        ax.grid(color=grid_color)
    
    # Set x-axis limits
    if x_lim is None:
        x_bot, x_top = np.amin(x), np.amax(x)
        if abs(x_bot - x_top) > 1e-66:
            x_range = x_top - x_bot
            ax.set_xlim(x_bot - 0.05 * x_range, x_top + 0.05 * x_range)
    else:
        ax.set_xlim(x_lim[0], x_lim[1])
    
    # Set y-axis limits
    if y_lim is None:
        y_bot, y_top = np.amin(y), np.amax(y)
        if abs(y_bot - y_top) > 1e-66:
            y_range = y_top - y_bot
            ax.set_ylim(y_bot - 0.05 * y_range, y_top + 0.05 * y_range)
    else:
        ax.set_ylim(y_lim[0], y_lim[1])
    
    ax.plot(x, y, linestyle=linestyle, color=color, marker=marker, ms=ms, 
            label=label, alpha=alpha)
    
    if fill_alpha > 0.0:
        ax.fill_between(x, fill_top, fill_bot, color=fill_color, alpha=fill_alpha)
    
    if label:
        ax.legend()

def image2d(ax, image, title='Image', x_label='Column', y_label='Row', 
            c_map='viridis', alpha=1.0,
            q_min=0.01, q_max=0.99, v_min=None, v_max=None, 
            scale='linear', invert=False, axes_off=False):
    '''
    Given ax for subplot and image, plot as required.
    Set upper and lower limits on color map according to chosen quantiles or using v_min and v_max.
    Scale color map linearly or logarithmically.
    Invert image (so the origin is in the bottom left corner) if required.
    '''
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    
    ax.set_xlim(0, image.shape[1])
    if invert:
        ax.set_ylim(0, image.shape[0])
    else:
        ax.set_ylim(image.shape[0], 0)
    
    # Determine normalisation using quantiles if explicit limits are not provided
    if v_min is None and v_max is None:
        if 0.0 < q_min < 1.0 and 0.0 < q_max < 1.0:
            v_min = np.quantile(image, q_min)
            v_max = np.quantile(image, q_max)
            norm = colors.LogNorm(vmin=v_min, vmax=v_max) if scale == 'log' \
                   else colors.Normalize(vmin=v_min, vmax=v_max)
        else:
            norm = None
    else:
        norm = colors.LogNorm(vmin=v_min, vmax=v_max) if scale == 'log' \
               else colors.Normalize(vmin=v_min, vmax=v_max)
    
    if axes_off:
        ax.axis('off')
    
    ax.imshow(image, norm=norm, cmap=c_map, alpha=alpha)

def scatter2d(ax, x, y, z=None, title='Scatter plot', x_label='x', y_label='y', 
              c_map='viridis', alpha=1.0, 
              x_lim=None, y_lim=None, norm=None, ax_equal=False, 
              marker='.', scale='linear', s=10,
              grid_color='g', label='', im=False, invert=False):
    '''
    Given ax for subplot, x and y arrays (and z array of required), 
    fill a scatter plot using markers with symbols and colors (sorry!) as required.
    Pick limits if these are not set.
    Scale color map if z array given according to chosen quartiles, using log or linear scaling.
    Invert the y axis (so the scatter plot is like an image) if required.
    Return an image object if requested so a color bar can be added.
    '''
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    
    if grid_color:
        ax.grid(color=grid_color)
    
    # Set x-axis limits
    if x_lim is None:
        x_bot, x_top = np.amin(x), np.amax(x)
        if abs(x_bot - x_top) > 1e-66:
            x_range = x_top - x_bot
            ax.set_xlim(x_bot - 0.05 * x_range, x_top + 0.05 * x_range)
    else:
        ax.set_xlim(x_lim[0], x_lim[1])
    
    # Set y-axis limits
    if y_lim is None:
        y_bot, y_top = np.amin(y), np.amax(y)
        if abs(y_bot - y_top) > 1e-66:
            y_range = y_top - y_bot
            if invert:
                ax.set_ylim(y_top + 0.05 * y_range, y_bot - 0.05 * y_range)
            else:
                ax.set_ylim(y_bot - 0.05 * y_range, y_top + 0.05 * y_range)
    else:
        if invert:
            ax.set_ylim(y_lim[1], y_lim[0])
        else:
            ax.set_ylim(y_lim[0], y_lim[1])
    
    if ax_equal:
        ax.axis('equal')
    
    if im:
        im_obj = ax.scatter(x, y, c=z, marker=marker, s=s, cmap=c_map, 
                            alpha=alpha, norm=norm, label=label)
        if label:
            ax.legend()
        return im_obj
    ax.scatter(x, y, c=z, marker=marker, s=s, cmap=c_map, 
               alpha=alpha, norm=norm, label=label)
    if label:
        ax.legend()

def make_disc_norm(data, n_colors=0, q_min=0.01, q_max=0.99, c_map_base='jet'):
    '''
    Make discrete (segmented) color map using chosen base cmap.
    Data assumed to have lowest value of zero.
    Define color table for plots.
    Set up normalisation to use segmented cmap in figures and limits and ticks for color bar. 
    '''
    if not np.issubdtype(data.dtype, np.bool_):
        v_min = np.quantile(data, q_min)
        v_max = np.quantile(data, q_max)
        if n_colors == 0:
            locator = ticker.AutoLocator()
            try:
                locator.create_dummy_axis()  
            except AttributeError:
                pass
            ticks = locator.tick_values(v_min, v_max)
            n_colors = len(ticks)
        step = (v_max - v_min) / n_colors
    else:
        v_min = 0.0
        v_max = 1.0
        step = 1.0
        n_colors = 2  # Boolean data: two colors
    
    c_map = plt.get_cmap(c_map_base, n_colors)
    color_table = c_map(np.linspace(0, 1, n_colors))
    c_map_seg = LinearSegmentedColormap.from_list("cmap_name", color_table, N=n_colors)
    limits = np.linspace(v_min - step/2.0, v_max + step/2.0, n_colors + 1)
    norm = colors.BoundaryNorm(limits, c_map_seg.N)
    ticks = np.linspace(v_min, v_max, n_colors)
    
    return n_colors, color_table, c_map_seg, norm, ticks

def make_cont_norm(data, q_min=0.01, q_max=0.99, scale='linear'):
    '''
    Make continuous color map using linear or log scale.
    Set up normalisation to use continuous cmap in figures and ticks for color bar. 
    '''
    v_min = np.quantile(data, q_min)
    v_max = np.quantile(data, q_max)
    norm = colors.LogNorm(vmin=v_min, vmax=v_max) if scale == 'log' \
           else colors.Normalize(vmin=v_min, vmax=v_max)
    
    locator = ticker.AutoLocator()
    try:
        locator.create_dummy_axis()  # For compatibility with some versions.
    except AttributeError:
        pass
    ticks = locator.tick_values(v_min, v_max)
    
    return norm, ticks


## Plot hyperspectral image at chosen wavenumber
-- takes FTIR, wavenumbers, bio_label as parameters instead of relying on a global variable

In [40]:
def plot_image(no_plots, k_image, FTIR, wavenumbers, bio_label):
    """
    Plot hyperspectral FTIR image at a chosen wavenumber.

    Parameters
    ----------
    no_plots : bool
        If True, the plot will *not* be displayed.
    k_image : the target wavenumber (in 1/cm) at which to display the image.
    FTIR : 3D nparray representing FTIR hyperspectral cube 
        with dimensions (n_rows, n_cols, n_wavenumbers).
    wavenumbers : 1D array of wavenumbers corresponding to the third dimension of FTIR.
    bio_label : biopsy label used for the plot title.
    
    Returns
    -------
    k_index : index of the wavenumber in `wavenumbers` closest to `k_image`.
    n_rows : no. rows in the FTIR image.
    n_cols : no. of columns in the FTIR image.
    """
    # Find index of the wavenumber closest to the desired k_image val.
    k_index = np.argmin(np.abs(k_image - wavenumbers))
    
    # get he image at the chosen wavenumber.
    FTIR_image = FTIR[:, :, k_index]
    n_rows, n_cols, n_ks = FTIR.shape
    k_min = wavenumbers[0]
    k_max = wavenumbers[-1]
    
    if not no_plots:
        print("\nPlot FTIR image at wavenumber", k_image, "1/cm")
        print(f"FTIR image has {n_rows} rows and {n_cols} columns")
        
        # create a figure 
        fig, ax = plt.subplots(1, 1, figsize=(2.5 * n_cols / 128, 2.5 * n_rows / 128))
        ax.set_title(f"FTIR image of biopsy {bio_label}")
        ax.imshow(FTIR_image, aspect='auto')
        ax.set_xlabel("Column")
        ax.set_ylabel("Row")
        ax.set_xlim(0, n_cols)
        ax.set_ylim(0, n_rows)
        plt.tight_layout()
        plt.show()
    
    return k_index, n_rows, n_cols


## Plot spectrum in chosen pixel

- Global variables FTIR and wavenumbers are now passed as parameters.
- ValueError raised if no region is found.

In [41]:
def plot_raw_spectrum(no_plots, row_T, col_T, row_NT, col_NT, FTIR, wavenumbers):
    """
    Plots raw FTIR spectrum from chosen pixel by selecting median pixel 
    from either the T or nT region.

    Parameters
    ----------
    no_plots : if True plot will *not* be displayed.
    row_T : row indices for transforming regions.
    col_T : column indices for transforming regions.
    row_NT : row indices for non-transforming regions.
    col_NT : column indices for non-transforming regions.
    FTIR : 3D FTIR hyperspectral cube with dims (rows, columns, wavenumbers).
    wavenumbers : array of wavenumbers corresponding to the third dimension of FTIR.
    
    Returns
    -------
    tuple
        (row_spect_raw, col_spect_raw, spectrum_raw) where:
          - row_spect_raw : the row index of the chosen pixel.
          - col_spect_raw : the column index of the chosen pixel.
          - spectrum_raw : the raw spectrum extracted from the chosen pixel.
              
    Raises ValueError if no T or NT regions are identified.
    """
    print("\nPlot raw spectrum: choose pixel")
    
    # Choose pixel in which spectrum will be plotted.
    if len(col_T) > 0:
        print(f"Min value of col_T: {np.amin(col_T)}, max value: {np.amax(col_T)}")
        print(f"Min value of row_T: {np.amin(row_T)}, max value: {np.amax(row_T)}")
        ind_spect = min(len(col_T), len(row_T)) // 2
        row_spect_raw = row_T[ind_spect]
        col_spect_raw = col_T[ind_spect]
    elif len(col_NT) > 0:
        print(f"Min value of col_NT: {np.amin(col_NT)}, max value: {np.amax(col_NT)}")
        print(f"Min value of row_NT: {np.amin(row_NT)}, max value: {np.amax(row_NT)}")
        ind_spect = min(len(col_NT), len(row_NT)) // 2
        row_spect_raw = row_NT[ind_spect]
        col_spect_raw = col_NT[ind_spect]
    else:
        raise ValueError("No transforming or non-transforming regions identified.")
    
    print(f"Plot raw spectrum in pixel, row {row_spect_raw} column {col_spect_raw}")
    
    # get the raw spectrum from selected pixel.
    spectrum_raw = FTIR[row_spect_raw, col_spect_raw, :]
    
    if not no_plots:
        fig, ax = plt.subplots(1, 1, figsize=(4, 3))
        ax.set_title(f"Spectrum from pixel ({row_spect_raw}, {col_spect_raw})")
        ax.plot(wavenumbers, spectrum_raw, color='r', linestyle='-')
        ax.set_xlabel("k (1/cm)")
        ax.set_ylabel("Absorbance")
        ax.grid(color='g')
        plt.tight_layout()
        plt.show()
    
    return row_spect_raw, col_spect_raw, spectrum_raw


## Look at corresponding processed spectrum
- now takes df and wavenumbers as parameters
- make sure only data from selected pixel are used:df_spectrum.loc[df['Image'] == num].drop(...) changed to df_spectrum.loc[df_spectrum['Image'] == num]
- using local df_spectrum instead of global df

In [42]:
def plot_proc_spectrum(no_plots, bio_label, row_spect_raw, col_spect_raw, spectrum_raw, df, wavenumbers):
    """
    Plot processed FTIR spectra for a particular pixel by comparing the raw 
    spectrum to one or more processed spectra extracted from df.
    
    Function creates 2 subplots:
      - left subplot shows entire raw spectrum alongside the processed spectra.
      - right subplot shows a zoomed-in view of the raw spectrum between the 
        processed spectral range.
    
    Parameters
    ----------
    no_plots : if True, the plots will *not* be displayed.
    bio_label : biopsy label corresponding to the pixel.
    row_spect_raw : the row index of the chosen pixel.
    col_spect_raw : column index of the chosen pixel.
    spectrum_raw : raw spectrum extracted from the chosen pixel.
    df : DF containing processed spectrum data. Expected to include columns:
        'biopsy', 'Image', 'outcome', 'Class', 'row', 'col', and additional columns 
        representing wavenumbers.
    wavenumbers : array of wavenumbers corresponding to the raw spectrum.
    
    Returns
    -------
    tuple
        (k_min_proc, k_max_proc, k_columns)
        where k_min_proc is the lowest processed wavenumber,
              k_max_proc is the highest processed wavenumber, and
              k_columns is the array of processed wavenumber values.
    """
    print(f"\nPlot processed spectrum in particular pixel, row {row_spect_raw} column {col_spect_raw}")
    
    # get processed spectral columns (wavenumbers) by dropping non-spectral columns.
    proc_columns = df.drop(['biopsy', 'Image', 'outcome', 'Class', 'row', 'col'], axis=1).columns
    k_columns = np.array([float(k) for k in proc_columns])
    nks_proc = len(k_columns)
    k_min_proc = k_columns[0]
    k_max_proc = k_columns[-1]
    
    # find the indices in the raw wavenumbers array corresponding to processed range.
    k_bot_ind = np.argmin(np.abs(wavenumbers - k_min_proc))
    k_top_ind = np.argmin(np.abs(wavenumbers - k_max_proc))
    
    # Filter df to the selected pixel.
    df_spectrum = df[(df['biopsy'] == bio_label) & 
                     (df['row'] == row_spect_raw) & 
                     (df['col'] == col_spect_raw)]
    
    image_num = df_spectrum['Image'].to_numpy()
    
    # get processed spectra from each image for the selected pixel.
    proc_spectra = []
    for num in image_num:
        # Filter within df_spectrum for current image no
        spec_row = df_spectrum.loc[df_spectrum['Image'] == num]
        # Drop non spectral columns and get the spectrum 
        proc_spectrum = spec_row.drop(['biopsy', 'Image', 'outcome', 'Class', 'row', 'col'], axis=1).to_numpy()[0, :]
        proc_spectra.append(proc_spectrum)
    
    colours = ['b', 'c', 'm', 'orange']
    n_colours = len(colours)
    
    if not no_plots:
        print("")
        fig, ax = plt.subplots(1, 2, figsize=(8, 3))
        
        # left subplot- full spectrum.
        ax[0].set_title(f"Spectra from pixel ({row_spect_raw}, {col_spect_raw})")
        ax[0].plot(wavenumbers, spectrum_raw, color='r', linestyle='-', label="Raw spectrum")
        for n, spect in enumerate(proc_spectra):
            ax[0].plot(k_columns, spect, linestyle='-', color=colours[n % n_colours],
                       label=f"Processed {image_num[n]}")
        ax[0].set_xlabel("k (1/cm)")
        ax[0].set_ylabel("Absorbance")
        ax[0].legend()
        ax[0].grid(color='g')
        
        # right subplot- zoomed raw spectrum in processed range.
        ax[1].set_title(f"Spectra from pixel ({row_spect_raw}, {col_spect_raw})")
        ax[1].plot(wavenumbers[k_bot_ind:k_top_ind], spectrum_raw[k_bot_ind:k_top_ind],
                   color='r', linestyle='-', label="Raw spectrum")
        for n, spect in enumerate(proc_spectra):
            ax[1].plot(k_columns, spect, linestyle='-', color=colours[n % n_colours],
                       label=f"Processed {image_num[n]}")
        ax[1].set_xlabel("k (1/cm)")
        ax[1].set_ylabel("Absorbance")
        ax[1].legend()
        ax[1].grid(color='g')
        
        plt.tight_layout()
        plt.show()
    
    return k_min_proc, k_max_proc, k_columns


## Plot FTIR image with non-transforming/transforming (N/T) pixels marked
- FTIR, wavenumbers, col_T, row_T, col_NT, row_NT used locally


In [43]:
def plot_FTIR_NTpix(no_plots, k_image, n_rows, n_cols, FTIR, wavenumbers, col_T, row_T, col_NT, row_NT):
    """
    Plot the FTIR hyperspectral image at a specified wavenumber with nt and t pixels marked.

    Parameters
    ----------
    no_plots : if True, the plots will *not* be displayed.
    k_image : target wavenumber (in 1/cm) at which to display the image.
    n_rows : number of rows in the FTIR image.
    n_cols : number of columns in the FTIR image.
    FTIR : 3D FTIR hyperspectral cube with dimensions (n_rows, n_cols, n_wavenumbers).
    wavenumbers : array of wavenumbers corresponding to the third dimension of FTIR.
    col_T : array of column indices for t pixels.
    row_T : array of row indices for t pixels.
    col_NT : array of column indices for nt pixels.
    row_NT : array of row indices for non-transforming pixels.

    Returns
    -------
    none
    """
    print("\nPlot FTIR image with N/T pixels marked")

    # Determine the index of the wavenumber closest to k_image.
    k_index = np.argmin(np.abs(k_image - wavenumbers))
    FTIR_image = FTIR[:, :, k_index]

    if not no_plots:
        print("")
        fig, ax = plt.subplots(1, 2, figsize=(5 * n_cols / 128, 2.5 * n_rows / 128))
        
        # left subplot- FTIR image with pixels marked.
        ax[0].imshow(FTIR_image)
        ax[0].set_title(f"k {int(k_image)} 1/cm, marked pixels")
        ax[0].set_xlabel("Column")
        ax[0].set_ylabel("Row")
        ax[0].scatter(col_T, row_T, color='r', alpha=0.3, s=1, label="Transforming")
        ax[0].scatter(col_NT, row_NT, color='orange', alpha=0.3, s=1, label="Non-transforming")
        ax[0].set_xlim(0, n_cols)
        ax[0].set_ylim(0, n_rows)
        ax[0].legend()
    
        # right subplot- FTIR image alone
        ax[1].imshow(FTIR_image)
        ax[1].set_title(f"FTIR image k {int(k_image)} 1/cm")
        ax[1].set_xlabel("Column")
        ax[1].set_ylabel("Row")
        ax[1].set_xlim(0, FTIR_image.shape[1])
        ax[1].set_ylim(0, FTIR_image.shape[0])
        
        plt.tight_layout()
        plt.show()
    
    return


## Make tissue mask
- FTIR and wavenumbers local
- avg absorabnce per pixel calucalted using np.mean so no need for loops
- threshold determined using quantile

In [44]:
def make_tissue_mask(no_plots, q_thresh, n_rows, n_cols, FTIR, wavenumbers):
    """
    Generate tissue mask from FTIR hyperspectral data by averaging absorbance 
    across wavenumber range and thresholding based on quantile.

    Parameters
    ----------
    no_plots : if True, the plot will *not* be displayed.
    q_thresh : quantile threshold (between 0 and 1) used to determine tissue mask.
    n_rows : no of rows in the FTIR image.
    n_cols : no of columns in the FTIR image.
    FTIR : 3D FTIR hyperspectral data with shape (n_rows, n_cols, n_wavenumbers).
    wavenumbers :  array of wavenumbers corresponding to the third dimension of FTIR.

    Returns
    -------
    tuple
        (k_low, k_high, in_tissue) where:
          - k_low : minimum wavenumber.
          - k_high : maximum wavenumber.
          - in_tissue : boolean mask of shape (n_rows, n_cols) where True indicates tissue.
    """
    print("\nMake tissue mask")
    
    # full range of wavenumbers.
    k_low = np.min(wavenumbers)
    k_high = np.max(wavenumbers)
    
    # Find indices corresponding to lowest and highest wavenumbers
    k_low_ind = np.argmin(np.abs(wavenumbers - k_low))
    k_high_ind = np.argmin(np.abs(wavenumbers - k_high))
    
    # Calc average absorbance for each pixel over the chosen wavenumber range
    # Replaces the nested loops with vector operation.
    sum_image = np.mean(FTIR[:, :, k_low_ind:k_high_ind], axis=2)
    
    # determine the threshold absorbance using specified quantile.
    a_thresh = np.quantile(sum_image, q_thresh)
    print(f"Quantile for threshold {q_thresh:.2f}, absorbance for threshold {a_thresh:.2e}")
    
    # Create the tissue mask: pixels with average absorbance above the threshold are considered tissue.
    in_tissue = sum_image > a_thresh
    
    if not no_plots:
        fig, ax = plt.subplots(1, 2, figsize=(5 * n_cols / 128, 2.5 * n_rows / 128))
        
        # Left subplot: Display the summed (average) absorbance image.
        ax[0].imshow(sum_image, cmap='viridis')
        ax[0].set_title("Summed image")
        ax[0].set_xlabel("Column")
        ax[0].set_ylabel("Row")
        ax[0].set_xlim(0, n_cols)
        ax[0].set_ylim(0, n_rows)
        
        # Right subplot: Display the tissue mask.
        ax[1].imshow(in_tissue, cmap='gray')
        ax[1].set_title("FTIR in tissue mask")
        ax[1].set_xlabel("Column")
        ax[1].set_ylabel("Row")
        ax[1].set_xlim(0, n_cols)
        ax[1].set_ylim(0, n_rows)
        
        plt.tight_layout()
        plt.show()
    
    return k_low, k_high, in_tissue


## Make N/T mask
- now has n_rows, n_cols, row_T, col_T, row_NT, and col_NT parameters

In [45]:
def make_NT_mask(no_plots, n_rows, n_cols, row_T, col_T, row_NT, col_NT):
    """
    Create mask indicating pixels belonging to t or nt regions.

    Parameters
    ----------
    no_plots :if True, the function will *not* display the plot.
    n_rows : no of rows in the FTIR image.
    n_cols : no of columns in the FTIR image.
    row_T : array of row indices for transforming pixels.
    col_T : array of column indices for transforming pixels.
    row_NT : array of row indices for non-transforming pixels.
    col_NT : array of column indices for non-transforming pixels.

    Returns
    -------
    NT_mask : bolean mask of shape (n_rows, n_cols) with True at positions corresponding
        to either t or nt pixels.
    """
    print("\nMake N/T mask")
    
    # Initialise bool mask with all False
    NT_mask = np.zeros((n_rows, n_cols), dtype=bool)
    
    # Mark T pixels
    NT_mask[row_T, col_T] = True 
    # Mark NT pixels
    NT_mask[row_NT, col_NT] = True 
    
    if not no_plots:
        fig, ax = plt.subplots(1, 1, figsize=(2.5 * n_cols / 128, 2.5 * n_rows / 128))
        ax.imshow(NT_mask, cmap='gray')
        ax.set_title("FTIR NT_mask")
        ax.set_xlabel("Column")
        ax.set_ylabel("Row")
        ax.set_xlim(0, n_cols)
        ax.set_ylim(0, n_rows)
        plt.tight_layout()
        plt.show()
    
    return NT_mask


## Plot spectra from N/T pixels 
- global variables arguments
- indices corresponding to processed wavenumber range calc using np.argmin applied to abs diff between wavenumber and the processed endpoints.
- raw spectra avg by summing over selected k range and dividing by no of pixels
- processed spectra from df averaged over the selected columns. 

In [46]:
def plot_pixel_NT_spectra(no_plots, k_min_proc, k_max_proc, k_columns, FTIR, NT_mask, 
                          wavenumbers, df, bio_label, row_spect_raw, col_spect_raw, col_T, col_NT):
    """
    Plot avg raw and processed spectra for pixels in NT mask
    
    Parameters
    ----------
    no_plots:if True, no plots are displayed.
    k_min_proc : minimum processed wavenumber.
    k_max_proc : maximum processed wavenumber.
    k_columns : array of processed wavenumbers (as floats).
    FTIR : FTIR hyperspectral cube.
    NT_mask :boolean mask indicating NT pixels.
    wavenumbers : array of raw wavenumbers.
    df : df containing processed spectrum data.
    bio_label : biopsy label.
    row_spect_raw : row index of the pixel chosen for raw spectrum.
    col_spect_raw : column index of the pixel chosen for the raw spectrum.
    col_T, col_NT : column indices for transforming and non-transforming pixels (used for averaging).
    
    Returns
    -------
    tuple
        (NT_mask_spect_raw, NT_mask_spect_proc)
    """
    print("\nPlot spectra from N/T pixels")
    
    # calc indices for the processed range in the wavenumbers array
    k_bot_ind = np.argmin(np.abs(wavenumbers - k_min_proc))
    k_top_ind = np.argmin(np.abs(wavenumbers - k_max_proc))
    
    # Compute avg raw spectrum for NT mask pixels
    NT_mask_spect_raw = FTIR[NT_mask, k_bot_ind:k_top_ind + 1]
    NT_mask_spect_raw = np.sum(NT_mask_spect_raw, axis=0)
    n_in_NT_mask = len(col_T) + len(col_NT)
    NT_mask_spect_raw = NT_mask_spect_raw / n_in_NT_mask

    # get processed spectral columns from the df
    proc_columns = df.drop(['biopsy', 'Image', 'outcome', 'Class', 'row', 'col'], axis=1).columns
    # Conv header values to floats.
    k_columns = np.array([float(k) for k in proc_columns])
    n_proc = len(k_columns)
    
    # filter df for the selected pixel.
    df_spectrum = df[(df['biopsy'] == bio_label) &
                     (df['row'] == row_spect_raw) &
                     (df['col'] == col_spect_raw)]
    image_num = df_spectrum['Image'].to_numpy()
    
    proc_spectra = []
    for num in image_num:
        spec_row = df_spectrum.loc[df_spectrum['Image'] == num]
        # drop non spectral cols
        proc_spectrum = spec_row.drop(['biopsy', 'Image', 'outcome', 'Class', 'row', 'col'], axis=1).to_numpy()[0, :]
        # Ensure the processed spectrum has the expected length.
        if proc_spectrum.shape[0] < n_proc:
            # Pad with NaN (or 0) to reach expected length.
            proc_spectrum = np.pad(proc_spectrum, (0, n_proc - proc_spectrum.shape[0]), mode='constant', constant_values=np.nan)
        elif proc_spectrum.shape[0] > n_proc:
            proc_spectrum = proc_spectrum[:n_proc]
        proc_spectra.append(proc_spectrum)
    
    # average the spectra over images available
    if proc_spectra:
        NT_mask_spect_proc = np.nanmean(proc_spectra, axis=0)
    else:
        NT_mask_spect_proc = np.zeros(n_proc)
    
    # plot
    if not no_plots:
        fig, ax = plt.subplots(1, 2, figsize=(8, 3))
        # Left subplot: full spectrum.
        ax[0].set_title(f"Spectra from pixel ({row_spect_raw}, {col_spect_raw})")
        ax[0].plot(wavenumbers, FTIR[row_spect_raw, col_spect_raw, :], color='r', linestyle='-', label="Raw spectrum")
        for n, spect in enumerate(proc_spectra):
            ax[0].plot(k_columns, spect, linestyle='-', color=['b', 'c', 'm', 'orange'][n % 4], label=f"Processed {image_num[n]}")
        ax[0].set_xlabel("k (1/cm)")
        ax[0].set_ylabel("Absorbance")
        ax[0].legend()
        ax[0].grid(color='g')
        
        # Right subplot zoomed view
        ax[1].set_title(f"Spectra from pixel ({row_spect_raw}, {col_spect_raw})")
        ax[1].plot(wavenumbers[k_bot_ind:k_top_ind], FTIR[row_spect_raw, col_spect_raw, k_bot_ind:k_top_ind], 
                   color='r', linestyle='-', label="Raw spectrum")
        for n, spect in enumerate(proc_spectra):
            ax[1].plot(k_columns, spect, linestyle='-', color=['b', 'c', 'm', 'orange'][n % 4], label=f"Processed {image_num[n]}")
        ax[1].set_xlabel("k (1/cm)")
        ax[1].set_ylabel("Absorbance")
        ax[1].legend()
        ax[1].grid(color='g')
        
        plt.tight_layout()
        plt.show()
    
    return NT_mask_spect_raw, NT_mask_spect_proc


## Histogram absorbances at selected wavenumbers in regions
- nested loop iterates over every pixel (using in_tissue and NT_mask) to get histograms for different regions

In [47]:
def absorbance_hist(no_plots, k_low, k_high, in_tissue, NT_mask, n_rows, n_cols, FTIR, wavenumbers):
    """
    Plot distribution of absorbance vals at selected wavenumbers for pixels 
    both in and out of tissue, and distinguish between pixels that are in 
    the NT regions versus those that aren't
    
    Computes histograms for:
      - Pixels outside the tissue mask.
      - Pixels inside tissue but not in the NT mask.
      - Pixels inside tissue that are in the NT mask.
    
    Parameters
    ----------
    no_plots : if True, the plots will not be displayed.
    k_low : lower bound of the selected wavenumber range.
    k_high : upper bound of the selected wavenumber range.
    in_tissue : boolean mask (shape: (n_rows, n_cols)) indicating tissue pixels.
    NT_mask : boolean mask (shape: (n_rows, n_cols)) indicating pixels in the NT regions.
    n_rows : number of rows in the FTIR image.
    n_cols : number of columns in the FTIR image.
    FTIR : 3D FTIR hyperspectral data with shape (n_rows, n_cols, n_wavenumbers).
    wavenumbers : array of raw wavenumbers corresponding to the third dimension of FTIR.
    
    Returns 
    None
    """
    print("\nPlot absorbance distribution at selected wavenumbers in/out of tissue, and N/T regions")
    
    # Determine indices in the wavenumbers array corresponding to selected range.
    k_low_ind = np.argmin(np.abs(wavenumbers - k_low))
    k_high_ind = np.argmin(np.abs(wavenumbers - k_high))
    
    print(f"Minimum absorbance {np.amin(FTIR[:, :, k_low_ind:k_high_ind]):.2e}")
    print(f"Maximum absorbance {np.amax(FTIR[:, :, k_low_ind:k_high_ind]):.2e}")
    
    n_ks = FTIR.shape[2]
    k_min = wavenumbers[0]
    k_max = wavenumbers[n_ks - 1]
    
    # histogram paramet set up
    n_bins = 215
    h_bot = -0.3
    h_top = 4.0
    bin_edges = np.linspace(h_bot, h_top, n_bins + 1)
    bin_centres = (bin_edges[:-1] + bin_edges[1:]) / 2.0
    bin_widths = (bin_edges[1] - bin_edges[0]) * np.ones(n_bins)
    
    # histograms for different regions
    hist_out_mask = np.zeros(n_bins)
    hist_in_tissue_NT = np.zeros(n_bins)
    hist_in_tissue_notNT = np.zeros(n_bins)
    
    # Loop over each pixel and get histograms based on tissue and NT masks
    for r in range(n_rows):
        for c in range(n_cols):
            # get absorbance values for the current pixel over the selected range.
            pixel_data = FTIR[r, c, k_low_ind:k_high_ind]
            hist, _ = np.histogram(pixel_data, bins=bin_edges)
            
            if in_tissue[r, c]:
                if NT_mask[r, c]:
                    hist_in_tissue_NT += hist
                else:
                    hist_in_tissue_notNT += hist
            else:
                hist_out_mask += hist

    if not no_plots:
        print("\nPlotting histogram of absorbance values...")
        fig = plt.figure(figsize=(5, 3))
        axA = fig.add_subplot(1, 1, 1)
        axA.set_title("Absorbance distribution")
        # Plot relative frequency histograms for each region.
        axA.bar(bin_centres, hist_out_mask / np.sum(hist_out_mask), bin_widths,
                linestyle='-', color='y', alpha=0.2, label="Not image")
        axA.bar(bin_centres, hist_in_tissue_notNT / np.sum(hist_in_tissue_notNT), bin_widths,
                linestyle='-', color='r', alpha=0.2, label="Image not N/T")
        axA.bar(bin_centres, hist_in_tissue_NT / np.sum(hist_in_tissue_NT), bin_widths,
                linestyle='-', color='b', alpha=0.2, label="Image N/T")
        axA.set_xlabel("Absorbance")
        axA.set_ylabel("Relative frequency")
        axA.set_yscale('log')
        axA.legend()
        axA.grid(color='g')
        plt.tight_layout()
        plt.show()
    
    return

## Plot spectra from pixels in N/T tissue and surrounds  
- Takes FTIR and wavenumbers
- function loops over each pixel to accumulate spectra for the different regions, divides by the number of pixels in each category to get average spectra.
- returns the avg spectra for pixels in tissie mask.

In [48]:
def plot_in_out_spectra(no_plots, in_tissue, NT_mask, k_min_proc, k_max_proc, n_rows, n_cols, FTIR, wavenumbers):
    """
    Compute and plot the average FTIR spectra over different regions of the image:
      - Pixels in the tissue mask ("in tissue"),
      - Pixels in tissue in the NT mask,
      - Pixels in tissue that are not in the NT mask,
      - Pixels outside tissue mask.

    For each, function averages the FTIR spectrum (across all wavenumbers)
    over all pixels in that category. Two plots are made:
      - Full-spectrum plot over the entire range of wavenumbers.
      - Zoomed-in plot over the processed wavenumber range [k_min_proc, k_max_proc].

    Parameters
    ----------
    no_plots : if True, the plots will not be displayed.
    in_tissue : boolean mask indicating pixels that are inside tissue.
    NT_mask : boolean mask indicating pixels that are in the NT regions.
    k_min_proc :minimum processed wavenumber (used for zooming in the plot).
    k_max_proc : maximum processed wavenumber (used for zooming in the plot).
    n_rows : number of rows in the FTIR image.
    n_cols : number of columns in the FTIR image.
    FTIR : 3D FTIR hyperspectral data.
    wavenumbers : array of wavenumbers corresponding to the third dimension of FTIR.

    Returns
    -------
    tuple of np.ndarray
        (in_tissue_spect, in_tissue_NT_spect, in_tissue_notNT_spect) where:
          - in_tissue_spect is average spectrum over all pixels inside tissue mask.
          - in_tissue_NT_spect is the average spectrum over pixels inside tissue in the NT mask.
          - in_tissue_notNT_spect is average spectrum over pixels inside tissue not in the NT mask.
    """
    print("\nPlot spectra in/out of tissue and N/T regions")
    
    n_ks = FTIR.shape[2]
    k_min = wavenumbers[0]
    k_max = wavenumbers[-1]
    
    # Determine indices in raw wavenumbers array corresponding to processed range
    k_bot_ind = np.argmin(np.abs(wavenumbers - k_min_proc))
    k_top_ind = np.argmin(np.abs(wavenumbers - k_max_proc))
    
    # accumulators for spectra and counters for pixels
    in_tissue_spect = np.zeros(n_ks)
    in_tissue_NT_spect = np.zeros(n_ks)
    in_tissue_notNT_spect = np.zeros(n_ks)
    out_mask_spect = np.zeros(n_ks)
    n_in_tissue_spect = 0
    n_in_tissue_NT_spect = 0
    n_in_tissue_notNT_spect = 0
    n_out_mask_spect = 0
    
    # loop over each pixel to accumulate spectra based on region.
    for r in range(n_rows):
        for c in range(n_cols):
            if in_tissue[r, c]:
                in_tissue_spect += FTIR[r, c, :]
                n_in_tissue_spect += 1
                if NT_mask[r, c]:
                    in_tissue_NT_spect += FTIR[r, c, :]
                    n_in_tissue_NT_spect += 1
                else:
                    in_tissue_notNT_spect += FTIR[r, c, :]
                    n_in_tissue_notNT_spect += 1
            else:
                out_mask_spect += FTIR[r, c, :]
                n_out_mask_spect += 1
    
    print(f"Number of pixels in in_tissue spectrum: {n_in_tissue_spect}")
    print(f"Number of pixels in in_tissue_NT spectrum: {n_in_tissue_NT_spect}")
    print(f"Number of pixels in in_tissue_notNT spectrum: {n_in_tissue_notNT_spect}")
    print(f"Number of pixels in out_mask spectrum: {n_out_mask_spect}")
    
    # Average the accumulated spectra over the number of pixels in each category.
    in_tissue_spect = in_tissue_spect / n_in_tissue_spect
    in_tissue_NT_spect = in_tissue_NT_spect / n_in_tissue_NT_spect
    in_tissue_notNT_spect = in_tissue_notNT_spect / n_in_tissue_notNT_spect
    if n_out_mask_spect > 1e-99:
        out_mask_spect = out_mask_spect / n_out_mask_spect
    else:
        out_mask_spect = 999 * out_mask_spect  # incase no out of tissue pixels.
    
    # plot
    if not no_plots:
        fig, ax = plt.subplots(1, 2, figsize=(7, 3))
        
        # Full-spectrum plot.
        ax[0].set_title("Spectra")
        ax[0].plot(wavenumbers, in_tissue_spect, color='c', linestyle='-', label="In mask")
        ax[0].plot(wavenumbers, in_tissue_NT_spect, color='b', linestyle='-', label="In mask NT")
        ax[0].plot(wavenumbers, in_tissue_notNT_spect, color='m', linestyle='-', label="In mask not NT")
        ax[0].plot(wavenumbers, out_mask_spect, color='r', linestyle='-', label="Out mask")
        ax[0].set_xlabel("k (1/cm)")
        ax[0].set_ylabel("Absorbance")
        ax[0].legend()
        ax[0].grid(color='g')
        
        # zoomed-in plot 
        ax[1].set_title("Spectra (Zoomed)")
        ax[1].plot(wavenumbers[k_bot_ind:k_top_ind], in_tissue_spect[k_bot_ind:k_top_ind],
                   color='c', linestyle='-', label="In mask")
        ax[1].plot(wavenumbers[k_bot_ind:k_top_ind], in_tissue_NT_spect[k_bot_ind:k_top_ind],
                   color='b', linestyle='-', label="In mask NT")
        ax[1].plot(wavenumbers[k_bot_ind:k_top_ind], in_tissue_notNT_spect[k_bot_ind:k_top_ind],
                   color='m', linestyle='-', label="In mask not NT")
        ax[1].plot(wavenumbers[k_bot_ind:k_top_ind], out_mask_spect[k_bot_ind:k_top_ind],
                   color='r', linestyle='-', label="Out mask")
        ax[1].set_xlabel("k (1/cm)")
        ax[1].set_ylabel("Absorbance")
        ax[1].legend()
        ax[1].grid(color='g')
        
        plt.tight_layout()
        plt.show()
    
    return in_tissue_spect, in_tissue_NT_spect, in_tissue_notNT_spect


## Plot image using range of wavenumbers summed
- again added parameters rather than use global vars.


In [49]:
def plot_multi_k_image(no_plots, all_plots, in_tissue_spect, NT_mask_spect_raw, NT_mask_spect_proc, 
                       k_min_proc, k_max_proc, n_rows, n_cols, FTIR, wavenumbers, k_columns):
    """
    Plot FTIR image and corresponding spectra over a range of wavenumbers.
    
    This:
      1. creates two spectral plots:
         - First plot shows the full pectrum data along with the processed spectra
           and marks specific image wavenumbers (k values) as markers
         - Second plot zooms in on the wavenumber range [k_min_proc, k_max_proc] of the raw spectrum.
      2. If requested (via `all_plots` ), displays a grid of FTIR images at a 
         selection of predefined wavenumbers.
    
    Parameters
    ----------
    no_plots : if True, the plots will not be displayed.
    all_plots : if True, an additional grid of FTIR images at selected k values is displayed.
    in_tissue_spect : averaged raw spectrum computed over pixels in tissue.
    NT_mask_spect_raw : averaged raw spectrum computed over NT-mask pixels.
    NT_mask_spect_proc : averaged processed spectrum computed over NT-mask pixels.
    k_min_proc : minimum processed wavenumber (used for zooming the spectral plot).
    k_max_proc : maximum processed wavenumber (used for zooming the spectral plot).
    n_rows : number of rows in the FTIR image.
    n_cols : number of columns in the FTIR image.
    FTIR : 3D FTIR hyperspectral cube with shape (n_rows, n_cols, n_wavenumbers).
    wavenumbers :array of raw wavenumbers corresponding to the third dimension of FTIR.
    k_columns : array of processed wavenumbers corresponding to the spectral columns in the Ddf
    
    Returns
    -------
    None
    """
    print("\nPlot image using a range of wavenumbers")
    
    # Get the full range of raw wavenumbers.
    n_ks = FTIR.shape[2]
    k_min = wavenumbers[0]
    k_max = wavenumbers[-1]
    
    # Determine indices for  processed wavenumber range.
    k_bot_ind = np.argmin(np.abs(wavenumbers - k_min_proc))
    k_top_ind = np.argmin(np.abs(wavenumbers - k_max_proc))
    
    # Define list of representative k values (in 1/cm) for which FTIR images will be displayed
    k_plot = [2600, 2700, 2800, 2900, 3000]
    n_plots = len(k_plot)
    k_plot_ind = [np.argmin(np.abs(k - wavenumbers)) for k in k_plot]
    
    # Define the layout for the grid of images.
    n_cols_plot = 3
    n_last_row = n_plots % n_cols_plot
    n_rows_plot = n_plots // n_cols_plot + int(n_last_row > 0)
    print(f"Number of columns in plot: {n_cols_plot}, number of rows: {n_rows_plot}")
    
    # If plotting enabled.
    if not no_plots:
        fig, ax = plt.subplots(1, 2, figsize=(7, 3))
        
        # left subplot: Full-spectrum plot.
        ax[0].set_title("Spectra with image k values")
        ax[0].plot(wavenumbers, in_tissue_spect, color='c', linestyle='-', label="Image")
        ax[0].plot(k_columns, NT_mask_spect_raw, color='r', linestyle='-', label="Raw")
        ax[0].plot(k_columns, NT_mask_spect_proc, color='b', linestyle='-', label="Processed")
        ax[0].plot(k_plot, np.zeros(n_plots), linestyle='', marker='^', color='k', label="Image k")
        ax[0].set_xlabel("k (1/cm)")
        ax[0].set_ylabel("Absorbance")
        ax[0].legend()
        ax[0].grid(color='g')
        
        # right subplot: Zoomed spectral plot.
        ax[1].set_title("Spectra with image k values")
        ax[1].plot(wavenumbers[k_bot_ind:k_top_ind], in_tissue_spect[k_bot_ind:k_top_ind],
                   color='c', linestyle='-', label="Image")
        ax[1].plot(k_columns, NT_mask_spect_raw, color='r', linestyle='-', label="Raw")
        ax[1].plot(k_columns, NT_mask_spect_proc, color='b', linestyle='-', label="Processed")
        ax[1].plot(k_plot, np.zeros(n_plots), linestyle='', marker='^', color='k', label="Image k")
        ax[1].set_xlabel("k (1/cm)")
        ax[1].set_ylabel("Absorbance")
        ax[1].set_xlim(k_min_proc, k_max_proc)
        ax[1].legend()
        ax[1].grid(color='g')
        
        plt.tight_layout()
        plt.show()
    
    # for more plots
    if all_plots:
        fig, ax = plt.subplots(n_rows_plot, n_cols_plot, 
                               figsize=(2.5 * n_cols_plot * n_cols / 128, 2.5 * n_rows_plot * n_rows / 128))
        print("")
        p = 0
        for r in range(n_rows_plot):
            for c in range(n_cols_plot):
                if p < n_plots:
                    ax[r, c].imshow(FTIR[:, :, k_plot_ind[p]])
                    ax[r, c].set_title(f"FTIR image, k {int(k_plot[p])} 1/cm")
                    if c == 0:
                        ax[r, c].set_ylabel("Row")
                    # Set x axis label on the bottom row.
                    if r == n_rows_plot - 1:
                        ax[r, c].set_xlabel("Column")
                    ax[r, c].set_xlim(0, n_cols)
                    ax[r, c].set_ylim(0, n_rows)
                else:
                    ax[r, c].axis('off')
                p += 1
        plt.tight_layout()
        plt.show()
    
    return


## Select wavenumbers

Choose wavenumbers to be used for nearest neighbour (NN) and multi-layer perceptron (MLP) analysis.

- wavenumbers, in_tissue_spect, k_columns, use_list parameters added
- f k_list is empty automatically generates list of wavenumbers using provided k_min and k_max values with a tolerance of 2
- indices corresponding to values in k_list are determined either by searching within a tolerance (if use_list True) or by using a linear spacing between the calculated min and max indices.

In [50]:
def select_wavenumbers(no_plots, NT_mask_spect_raw, NT_mask_spect_proc, 
                       k_list, k_min, k_max, k_min_proc, k_max_proc,
                       wavenumbers, in_tissue_spect, k_columns, use_list):
    """
    Select wavenumbers to be used for nn and mlp analysis.
    
    This function determines which wavenumbers (from the raw wavenumbers array) will be used for analysis.
    If a list of wavenumbers (k_list) is provided (i.e. non-empty), it is used as is; otherwise, the function
    automatically generates a list by finding indices within the range defined by k_min and k_max (using a tolerance
    of 2.0). Then it finds the corresponding indices (key_inds) in the raw wavenumbers array.
    
    Parameters
    ----------
    no_plots :if True, the function will not display the plot.
    NT_mask_spect_raw : averaged raw spectrum for NT mask pixels.
    NT_mask_spect_proc : aaveraged processed spectrum for NT mask pixels.
    k_list : list of wavenumbers (floats) to use for analysis. If empty, the list is generated automatically.
    k_min : minimum wavenumber to consider when automatically generating the list.
    k_max : maximum wavenumber to consider when automatically generating the list.
    k_min_proc : minimum processed wavenumber used for zooming in the spectral plot.
    k_max_proc : maximum processed wavenumber used for zooming in the spectral plot.
    wavenumbers : array of raw wavenumbers.
    in_tissue_spect : averaged raw spectrum computed over pixels inside tissue.
    k_columns : srray of processed wavenumbers corresponding to the spectral columns.
    use_list : if True, the provided k_list is used to find indices; otherwise, a linear spacing is used.
    
    Returns
    -------
    key_inds : np.ndarray of int
        Array of indices in the wavenumbers array corresponding to the selected wavenumbers.
    """
    print("\nSelect wavenumbers for NN and MLP analysis")
    
    # Determine the indices corresponding to the processed wavenumber range.
    k_bot_ind = np.argmin(np.abs(wavenumbers - k_min_proc))
    k_top_ind = np.argmin(np.abs(wavenumbers - k_max_proc))
    
    # If a list is provided, use its length, if not generate it automatically
    if len(k_list) > 0:
        n_key_list = len(k_list)
    else:
        k_ind_min = int(np.where(np.abs(wavenumbers - k_min) < 2.0)[0][0])
        k_ind_max = int(np.where(np.abs(wavenumbers - k_max) < 2.0)[0][0])
        n_key_list = k_ind_max - k_ind_min
        for n in range(k_ind_min, k_ind_max):
            k_list.append(wavenumbers[n])
    
    n_keys = n_key_list
    
    # Find indices (key_inds) in the raw wavenumbers array corresponding to the values in k_list
    key_inds = np.zeros(n_keys, dtype=int)
    if use_list:
        for n in range(n_keys):
            key_inds[n] = int(np.where(np.abs(wavenumbers - k_list[n]) < 2.0)[0][0])
    else:
        key_inds = np.linspace(k_ind_min, k_ind_max - 1, n_keys).astype(int)
    
    # Plot the spectra and mark the selected wavenumbers
    if not no_plots:
        fig, ax = plt.subplots(1, 2, figsize=(7, 3))
        
        # Full-spectrum plot.
        ax[0].set_title("Spectra + values used for k-means")
        ax[0].plot(wavenumbers, in_tissue_spect, color='c', linestyle='-', label="Image")
        ax[0].plot(k_columns, NT_mask_spect_raw, color='r', linestyle='-', label="Raw")
        ax[0].plot(k_columns, NT_mask_spect_proc, color='b', linestyle='-', label="Processed")
        ax[0].plot(wavenumbers[key_inds], np.zeros(n_keys), linestyle='', marker='^', 
                   color='k', label="Used for k-means")
        ax[0].set_xlabel("k (1/cm)")
        ax[0].set_ylabel("Absorbance")
        ax[0].legend()
        ax[0].grid(color='g')
        
        # Zoomed spectral plot.
        ax[1].set_title("Spectra + values used for k-means")
        ax[1].plot(wavenumbers[k_bot_ind:k_top_ind], in_tissue_spect[k_bot_ind:k_top_ind], 
                   color='c', linestyle='-', label="Image")
        ax[1].plot(k_columns, NT_mask_spect_raw, color='r', linestyle='-', label="Raw")
        ax[1].plot(k_columns, NT_mask_spect_proc, color='b', linestyle='-', label="Processed")
        ax[1].plot(wavenumbers[key_inds], np.zeros(n_keys), linestyle='', marker='^', 
                   color='k', label="Used for k-means")
        ax[1].set_xlabel("k (1/cm)")
        ax[1].set_ylabel("Absorbance")
        ax[1].set_xlim(k_min_proc, k_max_proc)
        ax[1].legend()
        ax[1].grid(color='g')
        
        plt.tight_layout()
        plt.show()
    
    return key_inds


## NN and MLP data: $X_D$ and $y_D$ arrays

The separation data for an individual biopsy are stored in the `X_D` array, with the first indices being the row and column positions of the pixel and the last index the wavenumber. The target values are in the `y_D` array, with the indices being the row and column umber of the pixel.  

- FTIR and wavenumbers parameters.
- number of spectral features (n_keys) computed from length of key_inds
- Normalisation methods:
    - Global Product: For each pixel, a boolean mask selects spectral values above norm_cut. Normalisation factor is computed as exponential of the average logarithm of these values.
    Global Sum: The spectrum is normalised by dividing by its sum.
    Local Ratio: Ratios of successive spectral values are calculated
    Local Difference: Differences between successive spectral values calculated
    Default: Raw spectral values (at the specified key indices) are used.

- Target array (y_D): for each pixel in tissue, target is set to NT_mask[r,c] * b_type.

In [51]:
def make_D_arrays(key_inds, in_tissue, NT_mask, global_prod, global_sum, local_ratio, local_diff, 
                  norm_cut, n_rows, n_cols, b_type, FTIR, wavenumbers):
    """
    Generate the data arrays for NN and MLP analysis.
    
    This produces the separation data array X_D and target array y_D for an
    individual biopsy. X_D stores for each pixel (row, col), the FTIR spectral features
    extracted using the indices provided in key_inds. The target values in y_D are set
    based on the NT_mask and scaled by b_type. Various normalisation or feature extraction
    techniques are applied depending on the flags:
    
      - global_prod: Global product normalisation,
      - global_sum: Global sum normalisation,
      - local_ratio: Local ratio features,
      - local_diff: Local difference features.
    If none of these flags are True, the raw FTIR values are used.
    
    Parameters
    ----------
    key_inds : indices of the wavenumbers to be used from the FTIR cube.
    in_tissue : boolean mask indicating which pixels belong to tissue.
    NT_mask : boolean mask indicating the nt pixels.
    global_prod :flag to use global product normalisation.
    global_sum : flag to use global sum normalisation.
    local_ratio : flag to use local ratio features.
    local_diff : flag to use local difference features.
    norm_cut : threshold for determining which spectral values are used in normalisation.
    n_rows : number of rows in the FTIR image.
    n_cols : number of columns in the FTIR image.
    b_type : biopsy type; used to scale the target values.
    FTIR : hyperspectral FTIR data cube.
    wavenumbers : array of wavenumbers corresponding to the third dimension of FTIR.
    
    Returns
    -------
    X_D : np.ndarray, shape (n_rows, n_cols, n_keys)
        Feature array containing the selected (and normalised/processed) spectral values.
    y_D : np.ndarray, shape (n_rows, n_cols)
        The target array, where each pixel’s value is NT_mask[r,c] * b_type.
    """
    # nnumber of keys to use.
    n_keys = len(key_inds)
    
    # initialise target array.
    y_D = np.zeros((n_rows, n_cols))
    
    # compute X_D.
    if global_prod:
        # Global product normalisation.
        norm_D_inds = np.zeros((n_rows, n_cols, n_keys), dtype=bool)
        norm_D = np.zeros((n_rows, n_cols))
        X_D = np.zeros((n_rows, n_cols, n_keys))
        for r in range(n_rows):
            for c in range(n_cols):
                if in_tissue[r, c]:
                    # Create a boolean mask for spectral values above norm_cut.
                    current_vals = FTIR[r, c, key_inds[:n_keys]]
                    mask = current_vals > norm_cut
                    norm_D_inds[r, c, :n_keys] = mask
                    # Compute the normalisation factor using only the values above norm_cut.
                    if np.sum(mask) > 0:
                        norm_D[r, c] = np.exp(np.sum(np.log(current_vals[mask])) / np.sum(mask))
                    else:
                        norm_D[r, c] = 1.0  # Fallback if no values exceed norm_cut.
                    X_D[r, c, :n_keys] = current_vals / norm_D[r, c]
                    y_D[r, c] = NT_mask[r, c] * b_type
    elif global_sum:
        # global sum normalisation.
        X_D = np.zeros((n_rows, n_cols, n_keys))
        for r in range(n_rows):
            for c in range(n_cols):
                if in_tissue[r, c]:
                    current_vals = FTIR[r, c, key_inds[:n_keys]]
                    norm_D = np.sum(current_vals)
                    if norm_D != 0:
                        X_D[r, c, :n_keys] = current_vals / norm_D
                    else:
                        X_D[r, c, :n_keys] = current_vals
                    y_D[r, c] = NT_mask[r, c] * b_type
    elif local_ratio:
        # Local ratio features.
        X_D = np.zeros((n_rows, n_cols, n_keys))
        for r in range(n_rows):
            for c in range(n_cols):
                if in_tissue[r, c]:
                    # Calc ratio of successive values.
                    # For indices 0 to n_keys-2, ratio = FTIR[r,c, key_inds[i+1]] / FTIR[r,c, key_inds[i]]
                    X_D[r, c, :n_keys - 1] = FTIR[r, c, key_inds[1:n_keys]] / FTIR[r, c, key_inds[:n_keys - 1]]
                    X_D[r, c, n_keys - 1] = 0.0  # Last feature is set to zero.
                    y_D[r, c] = NT_mask[r, c] * b_type
    elif local_diff:
        # Local difference features.
        X_D = np.zeros((n_rows, n_cols, n_keys))
        for r in range(n_rows):
            for c in range(n_cols):
                if in_tissue[r, c]:
                    # Compute difference between successive values.
                    X_D[r, c, :n_keys - 1] = FTIR[r, c, key_inds[1:n_keys]] - FTIR[r, c, key_inds[:n_keys - 1]]
                    X_D[r, c, n_keys - 1] = 0.0
                    y_D[r, c] = NT_mask[r, c] * b_type
    else:
        # No additional processing; use the raw FTIR values at the selected indices.
        X_D = np.zeros((n_rows, n_cols, n_keys))
        for r in range(n_rows):
            for c in range(n_cols):
                if in_tissue[r, c]:
                    X_D[r, c, :n_keys] = FTIR[r, c, key_inds[:n_keys]]
                    y_D[r, c] = NT_mask[r, c] * b_type
    return X_D, y_D


## Plot X_D as a set of images

Works only for one biopsy.

- For each slice (indexed by k_ind), tcomputes the quantile based limits (v_min and v_max) for the colour scale of that slice (unless a global value being used)
- image2d function is called twice on each subplot: first to show the feature image from X_D_arr, then to overlay the corresponding y_D_arr.
- 1D vs 2D axes: if only one row of subplots is created, ax is a 1D array, and the indexing is adjusted.

In [52]:
def plot_X_D_arr(all_plots, X_D_arr, y_D_arr, n_rows, n_cols):
    """
    Plot the X_D and y_D arrays as a set of images for one biopsy.

    The function selects a range of wavenumber indices from the feature array X_D_arr
    and displays each slice as an image, overlaying the corresponding y_D_arr (as a 
    semi-transparent grayscale image) to indicate the segmentation (e.g., tissue mask).
    
    Parameters
    ----------
    all_plots : if True, images will be plotted.
    X_D_arr : feature array with shape (n_rows, n_cols, n_keys).
    y_D_arr :target array with shape (n_rows, n_cols).
    n_rows : number of rows in the FTIR image.
    n_cols : int
        Number of columns in the FTIR image.
        
    Returns
    -------
    None
    """
    if all_plots:
        print("\nPlot X_D and y_D arrays")
        use_global_v = False
        
        #range of wavenumber indices to plot.
        k_ind_min = 0
        k_ind_max = 10
        # Step length (indices) for plots.
        k_ind_step = 2
        n_plots = (k_ind_max - k_ind_min) // k_ind_step
        n_cols_plot = 4
        n_rows_plot = n_plots // n_cols_plot + (n_plots % n_cols_plot > 0)
        
        print("")
        # Create subplots; figure size scaled relative to image dimensions.
        fig, ax = plt.subplots(n_rows_plot, n_cols_plot, 
                               figsize=(2.5 * n_cols_plot * n_cols / 128, 2.5 * n_rows_plot * n_rows / 128))
        
        q_min = 0.01
        q_max = 0.99
        
        # option to compute global v_min and v_max over selected slices
        if use_global_v:
            v_min = np.quantile(X_D_arr[:, :, k_ind_min:k_ind_max], q_min)
            v_max = np.quantile(X_D_arr[:, :, k_ind_min:k_ind_max], q_max)
        
        k_ind = k_ind_min
        if n_rows_plot > 1:
            for r in range(n_rows_plot):
                for c in range(n_cols_plot):
                    if k_ind < k_ind_max:
                        if not use_global_v:
                            v_min = np.quantile(X_D_arr[:, :, k_ind], q_min)
                            v_max = np.quantile(X_D_arr[:, :, k_ind], q_max)
                        # plot feature image
                        image2d(ax[r, c], X_D_arr[:, :, k_ind], title=f"k index = {k_ind}", 
                                x_label='Column', y_label='Row', invert=True,
                                c_map='viridis', v_min=v_min, v_max=v_max, scale='linear')
                        # overlay target array with transparency
                        image2d(ax[r, c], y_D_arr, title=f"k index = {k_ind}", 
                                x_label='Column', y_label='Row', invert=True,
                                c_map='Grays', alpha=0.3)
                        k_ind += k_ind_step
                    else:
                        ax[r, c].axis('off')
        else:
            for i in range(n_cols_plot):
                if k_ind < k_ind_max:
                    if not use_global_v:
                        v_min = np.quantile(X_D_arr[:, :, k_ind], q_min)
                        v_max = np.quantile(X_D_arr[:, :, k_ind], q_max)
                    image2d(ax[i], X_D_arr[:, :, k_ind], title=f"k index = {k_ind}", 
                            x_label='Column', y_label='Row', invert=True,
                            c_map='viridis', v_min=v_min, v_max=v_max, scale='linear')
                    image2d(ax[i], y_D_arr, title=f"k index = {k_ind}", 
                            x_label='Column', y_label='Row', invert=True,
                            c_map='Grays', alpha=0.3)
                    k_ind += k_ind_step
                else:
                    ax[i].axis('off')
                    
        plt.tight_layout()
        plt.show()
    return


## NN analysis

This version treats N and T together.

Data stored in array `nn_data` with first index idicating the pixel to which it belongs and the second the wavenumber. The rows and columns the pixel index refers to are stored in the arrays `row_nn_pix` and `col_nn_pix`, respectively. 

- add FTIR, wavenumbers, bio_list, bio_type, X_D_bio, y_D_bio, etc
- function loops over each biopsy and each pixel (only if in tissue) to build feature matrix X_NN from X_D_bio and target vector y_NN from y_D_bio. Optional positional features appended.
- 

In [53]:
def NN_analysis_all(no_plots, key_inds, nns, nns_plot, n_bio_list, n_rows_bio, n_cols_bio,
                    in_tissue_bio, FTIR, wavenumbers, bio_list, bio_type, X_D_bio, y_D_bio):
    """
    nn analysis on multiple biopsies, treating nt and t pixels together.

    For each biopsy, function constructs a feature matrix (X_NN) for all pixels in the tissue,
    where each pixel's features are extracted from X_D_bio (array of shape 
    (n_bio_list, n_rows, n_cols, n_keys)). Corresponding target values are stored in y_D_bio.
    Positional information can be appended (controlled by use_pos flag).

    The function then fits a NN model on X_NN and retrieves the nearest neighbors for 
    each pixel. Various diagnostic plots are generated to check for any issues in the input data or 
    in the NN outputs.

    Parameters
    ----------
    no_plots : if True, no plots will be displayed.
    key_inds :indices of the features (wavenumbers) to be used.
    nns : number of nearest neighbors to compute.
    nns_plot : number of nearest neighbors (beyond the home pixel) to plot for each pixel.
    n_bio_list : number of biopsies.
    n_rows_bio : nNumber of rows for each biopsy.
    n_cols_bio : number of columns for each biopsy.
    in_tissue_bio : boolean array of shape (n_bio_list, max_rows, max_cols) indicating tissue pixels.
    FTIR : FTIR hyperspectral cube (used here only for shape information) with shape (n_rows, n_cols, n_wavenumbers).
    wavenumbers : array of wavenumbers.
    bio_list : list of biopsy identifiers.
    bio_type : array of biopsy types (one per biopsy) used for filtering.
    X_D_bio : feature data array of shape (n_bio_list, max_rows, max_cols, n_keys).
    y_D_bio : target data array of shape (n_bio_list, max_rows, max_cols).

    Returns
    -------
    None
    """
    print("\nNearest neighbors study")

    # Get FTIR shape info
    n_ks = FTIR.shape[2]
    k_min = wavenumbers[0]
    k_max = wavenumbers[-1]
    n_keys = len(key_inds)

    # check that n_bio_list is consistent with bio_list
    n_bio_list = len(bio_list)

    # option to include position features
    use_pos = False
    pos_wgt = 1.0
    if use_pos:
        n_NN = n_bio_list * (n_keys + 2)
    else:
        n_NN = n_bio_list * n_keys

    # calc total number of pixels across all biopsies
    n_pixels = sum(n_rows_bio[i] * n_cols_bio[i] for i in range(n_bio_list))
    n_pixels = int(n_pixels)

    # Initialize arrays for NN analysis.
    X_NN = np.zeros((n_pixels, n_NN))
    r_NN = np.zeros(n_pixels)
    c_NN = np.zeros(n_pixels)
    y_NN = np.zeros(n_pixels, dtype=int)
    b_NN = np.zeros(n_pixels, dtype=int)

    n_NN_pix = 0
    # loop through each biopsy and each pixel.
    for nb in range(n_bio_list):
        for r in range(n_rows_bio[nb]):
            for c in range(n_cols_bio[nb]):
                if not in_tissue_bio[nb, r, c]:
                    continue
                # Copy the selected features (from X_D_bio) into the NN feature matrix.
                X_NN[n_NN_pix, 0:n_keys] = X_D_bio[nb, r, c, 0:n_keys]
                if use_pos:
                    X_NN[n_NN_pix, n_keys] = pos_wgt * r / n_rows_bio[nb]
                    X_NN[n_NN_pix, n_keys + 1] = pos_wgt * c / n_cols_bio[nb]
                r_NN[n_NN_pix] = r
                c_NN[n_NN_pix] = c
                y_NN[n_NN_pix] = y_D_bio[nb, r, c]
                b_NN[n_NN_pix] = nb
                n_NN_pix += 1

    # Trim arrays to only the valid pixels.
    X_NN = X_NN[:n_NN_pix, :]
    r_NN = r_NN[:n_NN_pix]
    c_NN = c_NN[:n_NN_pix]
    y_NN = y_NN[:n_NN_pix]
    b_NN = b_NN[:n_NN_pix]

    # Validate input positions per biopsy.
    for nb in range(n_bio_list):
        bool_bio = (b_NN == bio_type[nb])
        if np.sum(bool_bio) < 1:
            continue
        if (np.amax(r_NN[bool_bio]) > n_rows_bio[nb] or np.amax(c_NN[bool_bio]) > n_cols_bio[nb]):
            fig = plt.figure(figsize=(5, 3))
            axA = fig.add_subplot(1, 1, 1)
            axA.set_title(f"Bad input col/row, biopsy {bio_list[nb]}")
            axA.set_xlabel("Col")
            axA.set_ylabel("Row")
            axA.scatter(c_NN[bool_bio], r_NN[bool_bio], color='c', s=1.0)
            plt.tight_layout()
            plt.show()

    # Find nearest neighbors of pixels in N/T mask
    neighbors = NearestNeighbors(n_neighbors=nns)
    neighbors_fit = neighbors.fit(X_NN)
# Get distances to nearest neighbour pixels (in k space!) and indices of nearest neighbour 
# pixels. Zeroth distance/index is the home pixel itself, first is nearest neighbour, second 
# next-nearest... E.g. distance[2000, 1] is the distance to the pixel nearest pixel 2000 and 
# indices[2000, 1] is the index of the pixel nearest pixel 2000. (Here, 2000 is the index of 
# the pixel in the X_NN, y_NN, r_NN etc. arrays)

    distances, indices = neighbors_fit.kneighbors(X_NN)

    # for each, check NN outputs for pixels with target value 1 (N) and 2 (T)
    for nb in range(n_bio_list):
        bool_bio_N = np.logical_and(b_NN == bio_type[nb], y_NN == 1)
        bool_bio_T = np.logical_and(b_NN == bio_type[nb], y_NN == 2)
        if np.sum(bool_bio_N) < 1:
            continue
        for n in range(1, nns_plot + 1):
            if (np.amax(r_NN[indices[bool_bio_N, n]]) > n_rows_bio[nb] or 
                np.amax(c_NN[indices[bool_bio_N, n]]) > n_cols_bio[nb]):
                fig = plt.figure(figsize=(5, 3))
                axA = fig.add_subplot(1, 1, 1)
                axA.set_title(f"Bad output col/row, biopsy {bio_list[nb]}, neighbour {n}")
                axA.set_xlabel("Col")
                axA.set_ylabel("Row")
                axA.scatter(c_NN[bool_bio_N], r_NN[bool_bio_N], color='c', s=1.0)
                axA.scatter(c_NN[indices[bool_bio_N, n]], r_NN[indices[bool_bio_N, n]], s=1.0,
                            label="NN" + str(n))
                plt.tight_layout()
                plt.show()
        if np.sum(bool_bio_T) < 1:
            continue
        for n in range(1, nns_plot + 1):
            if (np.amax(r_NN[indices[bool_bio_T, n]]) > n_rows_bio[nb] or 
                np.amax(c_NN[indices[bool_bio_T, n]]) > n_cols_bio[nb]):
                fig = plt.figure(figsize=(5, 3))
                axA = fig.add_subplot(1, 1, 1)
                axA.set_title(f"Bad output col/row, biopsy {bio_list[nb]}, neighbour {n}")
                axA.set_xlabel("Col")
                axA.set_ylabel("Row")
                axA.scatter(c_NN[bool_bio_T], r_NN[bool_bio_T], color='c', s=1.0)
                axA.scatter(c_NN[indices[bool_bio_T, n]], r_NN[indices[bool_bio_T, n]], s=1.0,
                            label="NN" + str(n))
                plt.tight_layout()
                plt.show()

    # Sort by the distance to their first nearest neighbour (excl self)
    i_sort = np.argsort(distances[:, 1])
    sorted_dists = distances[i_sort]
    pix_arr = np.linspace(0, n_NN_pix, n_NN_pix)

    if not no_plots:
        fig = plt.figure(figsize=(5, 3))
        axA = fig.add_subplot(1, 2, 1)
        axA.plot(pix_arr, sorted_dists[:, 1], color='r')
        axA.set_title("Sorted distance to NN")
        axA.set_xlabel("Pixel")
        axA.set_ylabel("Distance")
        axA.grid(color='g')

        axB = fig.add_subplot(1, 2, 2)
        axB.plot(pix_arr, sorted_dists[:, 1], color='r')
        axB.set_title("Sorted distance to NN")
        axB.set_xlabel("Pixel")
        axB.set_ylabel("Distance")
        axB.grid(color='g')
        axB.set_ylim(np.quantile(sorted_dists[:, 1], 0.01), np.quantile(sorted_dists[:, 1], 0.99))
        plt.tight_layout()
        plt.show()

        # For each biopsy, plot scatter plots showing tissue pixels and the NN of N/T pixels.
        for nb in range(n_bio_list):
            bool_bio = (b_NN == nb)
            bool_bio_NT = np.logical_and(b_NN == nb, y_NN == 1)
            fig = plt.figure(figsize=(5 * n_cols_bio[nb] / 128, 2.5 * n_rows_bio[nb] / 128))
            axA = fig.add_subplot(1, 2, 1)
            axA.set_title(f"N/T mask biopsy {bio_list[nb]}")
            axA.set_xlabel("Col")
            axA.set_ylabel("Row")
            axA.scatter(c_NN[bool_bio], r_NN[bool_bio], s=1.0, color='c', label="Tissue")
            axA.scatter(c_NN[bool_bio_NT], r_NN[bool_bio_NT], s=1.0, color='b', label="N/T")
            axA.legend(loc=3)
            axA.set_xlim(0, n_cols_bio[nb])
            axA.set_ylim(0, n_rows_bio[nb])
            axA.axis('equal')

            axB = fig.add_subplot(1, 2, 2)
            axB.set_title(f"NN to N/T mask {bio_list[nb]}")
            axB.set_xlabel("Col")
            axB.set_ylabel("Row")
            axB.scatter(c_NN[bool_bio], r_NN[bool_bio], color='c', label="Tissue", s=1.0)
            for n in range(1, nns_plot + 1):
                axB.scatter(c_NN[indices[bool_bio_NT, n]], r_NN[indices[bool_bio_NT, n]], s=1.0,
                            label="NN" + str(n))
            axB.legend(loc=3)
            axB.set_xlim(0, n_cols_bio[nb])
            axB.set_ylim(0, n_rows_bio[nb])
            axB.axis('equal')

            plt.tight_layout()
            plt.show()

    return



## NN analysis

Treats N and T separately. That is, all the data provided contain either N or T cells, not both. 

Data are stored in array `X_NN` with first index indicating the pixel to which they belong and the second the wavenumber. The rows and columns the pixel index refers to are stored in the arrays `r_NN` and `c_NN` respectively. The value of `y_NN` indicates the target for that pixel, whether it is tissue (0), N (1) or (2), while `b_NN` is the biopsy index.

- All inputs such as the arrays (X_D_bio_here, y_D_bio_here), tissue masks (in_tissue_here), image dimensions (n_rows_here, n_cols_here), and biopsy identifiers (bio_list_here) passed as parameters
- distinguishes pixels by comparing y_NN with b_type. Only pixels with the target matching b_type (which represents the cell type of interest, either N or T) are considered when plotting nearest neighbors

In [54]:
def NN_analysis_one(no_plots, key_inds, nns, nns_plot,
                    n_bio_here, n_rows_here, n_cols_here, in_tissue_here, 
                    X_D_bio_here, y_D_bio_here, bio_list_here, b_type):
    """
    Perform nn analysis on data from one cell type (either N or T) for one or
    more biopsies. Function constructs a feature matrix X_NN for all pixels in the biopsies
    (only for those pixels that are in tissue) using the features from X_D_bio_here. Also builds arrays
    r_NN and c_NN (storing the row and column indices), and y_NN and b_NN (storing the target
    value and biopsy index).

    NN analysis is then performed using scikit Nearestneighbors. Diagnostic plots check for any errors in the input coordinates as well as in the NN outputs. Note that the home pixel is
    always included as the first (zeroth) neighbour.

    Parameters
    ----------
    no_plots : if True, no plots will be displayed.
    key_inds : indices of the spectral features (wavenumbers) to be used.
    nns : number of nearest neighbors to compute.
    nns_plot : number of nearest neighbors (beyond the home pixel) to plot.
    n_bio_here : number of biopsies included in this analysis.
    n_rows_here : for each biopsy, the number of rows in the image.
    n_cols_here : for each biopsy, the number of columns in the image.
    in_tissue_here : boolean mask indicating for each biopsy which pixels are in tissue.
    X_D_bio_here : feature data array for each biopsy.
    y_D_bio_here : target data array for each biopsy. For this analysis, all values should be either 1 (for N)
        or 2 (for T); the parameter b_type indicates which.
    bio_list_here : list of biopsy identifiers.
    b_type : cell type value to assign in y_NN (1 for N, 2 for T).

    Returns
    -------
    None
    """
    print("\nNearest neighbors study")
    
    n_keys = len(key_inds)
    
    # Optionally, include positional features.
    use_pos = False
    pos_wgt = 1.0
    if use_pos:
        n_NN = n_bio_here * (n_keys + 2)
    else:
        n_NN = n_bio_here * n_keys

    # Compute total number of pixels across all biopsies.
    n_pixels = int(np.sum(n_rows_here * n_cols_here))
    
    # Initialize arrays for the NN analysis.
    X_NN = np.zeros((n_pixels, n_NN))
    r_NN = np.zeros(n_pixels)
    c_NN = np.zeros(n_pixels)
    y_NN = np.zeros(n_pixels, dtype=int)
    b_NN = np.zeros(n_pixels, dtype=int)
    
    n_NN_pix = 0
    # Loop through each biopsy and each pixel.
    for nb in range(n_bio_here):
        for r in range(n_rows_here[nb]):
            for c in range(n_cols_here[nb]):
                if not in_tissue_here[nb, r, c]:
                    continue
                # Copy features from X_D_bio_here.
                X_NN[n_NN_pix, 0:n_keys] = X_D_bio_here[nb, r, c, 0:n_keys]
                if use_pos:
                    X_NN[n_NN_pix, n_keys] = pos_wgt * r / n_rows_here[nb]
                    X_NN[n_NN_pix, n_keys + 1] = pos_wgt * c / n_cols_here[nb]
                r_NN[n_NN_pix] = r
                c_NN[n_NN_pix] = c
                y_NN[n_NN_pix] = y_D_bio_here[nb, r, c]
                b_NN[n_NN_pix] = nb
                n_NN_pix += 1

    # Trim arrays to contain only the valid pixels.
    X_NN = X_NN[:n_NN_pix, :]
    r_NN = r_NN[:n_NN_pix]
    c_NN = c_NN[:n_NN_pix]
    y_NN = y_NN[:n_NN_pix]
    b_NN = b_NN[:n_NN_pix]
    
    # Check that pixel coordinates are within expected bounds.
    for nb in range(n_bio_here):
        bool_bio = (b_NN == nb)
        if np.amax(r_NN[bool_bio]) > n_rows_here[nb] or np.amax(c_NN[bool_bio]) > n_cols_here[nb]:
            fig = plt.figure(figsize=(5, 3))
            axA = fig.add_subplot(1, 1, 1)
            axA.set_title(f"Bad input col/row, biopsy {bio_list_here[nb]}")
            axA.set_xlabel("Col")
            axA.set_ylabel("Row")
            axA.scatter(c_NN[bool_bio], r_NN[bool_bio], color='c', s=1.0)
            plt.tight_layout()
            plt.show()
    
    # Fit the Nearest neighbors model.
    neighbors = NearestNeighbors(n_neighbors=nns)
    neighbors_fit = neighbors.fit(X_NN)
    distances, indices = neighbors_fit.kneighbors(X_NN)
    
    # For each biopsy, consider only pixels whose target equals b_type.
    for nb in range(n_bio_here):
        bool_bio_NT = np.logical_and(b_NN == nb, y_NN == b_type)
        if np.sum(bool_bio_NT) < 1:
            continue
        for n in range(1, nns_plot + 1):
            # Check neighbour coords within bounds.
            if (np.amax(r_NN[indices[bool_bio_NT, n]]) > n_rows_here[nb] or 
                np.amax(c_NN[indices[bool_bio_NT, n]]) > n_cols_here[nb]):
                fig = plt.figure(figsize=(5, 3))
                axA = fig.add_subplot(1, 1, 1)
                axA.set_title(f"Bad output col/row, biopsy {bio_list_here[nb]}, neighbour {n}")
                axA.set_xlabel("Col")
                axA.set_ylabel("Row")
                axA.scatter(c_NN[bool_bio], r_NN[bool_bio], color='c', s=1.0)
                axA.scatter(c_NN[indices[bool_bio_NT, n]], r_NN[indices[bool_bio_NT, n]], 
                            s=1.0, label="NN" + str(n))
                plt.tight_layout()
                plt.show()
    
    # Sort pixels by the distance to their first nearest neighbour (ignoring the zero self-distance).
    i_sort = np.argsort(distances[:, 1])
    sorted_dists = distances[i_sort]
    pix_arr = np.linspace(0, n_NN_pix, n_NN_pix)
    
    if not no_plots:
        fig = plt.figure(figsize=(5, 3))
        axA = fig.add_subplot(1, 2, 1)
        axA.plot(pix_arr, sorted_dists[:, 1], color='r')
        axA.set_title("Sorted distance to NN")
        axA.set_xlabel("Pixel")
        axA.set_ylabel("Distance")
        axA.grid(color='g')
        
        axB = fig.add_subplot(1, 2, 2)
        axB.plot(pix_arr, sorted_dists[:, 1], color='r')
        axB.set_title("Sorted distance to NN")
        axB.set_xlabel("Pixel")
        axB.set_ylabel("Distance")
        axB.grid(color='g')
        axB.set_ylim(np.quantile(sorted_dists[:, 1], 0.01), np.quantile(sorted_dists[:, 1], 0.99))
        plt.tight_layout()
        plt.show()
        
        # For each biopsy produce scatter plots of tissue pixels and NN of pixels with target b_type
        for nb in range(n_bio_here):
            bool_bio = (b_NN == nb)
            bool_bio_NT = np.logical_and(b_NN == nb, y_NN == b_type)
            fig = plt.figure(figsize=(5 * n_cols_here[nb] / 128, 2.5 * n_rows_here[nb] / 128))
            axA = fig.add_subplot(1, 2, 1)
            axA.set_title(f"N/T mask {bio_list_here[nb]}")
            axA.set_xlabel("Col")
            axA.set_ylabel("Row")
            axA.scatter(c_NN[bool_bio], r_NN[bool_bio], s=1.0, color='c', label="Tissue")
            axA.scatter(c_NN[bool_bio_NT], r_NN[bool_bio_NT], s=1.0, color='b', label="N/T")
            axA.legend(loc=3)
            axA.set_xlim(0, n_cols_here[nb])
            axA.set_ylim(0, n_rows_here[nb])
            axA.axis('equal')
            
            axB = fig.add_subplot(1, 2, 2)
            axB.set_title(f"NN to N/T mask {bio_list_here[nb]}")
            axB.set_xlabel("Col")
            axB.set_ylabel("Row")
            axB.scatter(c_NN[bool_bio], r_NN[bool_bio], color='c', label="Tissue", s=1.0)
            for n in range(1, nns_plot + 1):
                axB.scatter(c_NN[indices[bool_bio_NT, n]], r_NN[indices[bool_bio_NT, n]], 
                            s=1.0, label="NN" + str(n))
            axB.legend(loc=3)
            axB.set_xlim(0, n_cols_here[nb])
            axB.set_ylim(0, n_rows_here[nb])
            axB.axis('equal')
            
            plt.tight_layout()
            plt.show()
    
    return


## Split data into training and test samples and scale

Note that each biopsy has different numbers of rows and columns.

The training set consists of a proportion of the pixels. How can the information about the location of the pixels be retained? Need to do the train test split "by hand" rather than using the sklearn routine!


- requires bio_list
- a random matrix (splitter) is generated to decide whether each tissue pixel is assigned to the training set or to the test set. The pixel’s row, column, and biopsy index are stored alongside the feature vector and target value.
- Arrays preallocated based on the total number of pixels (across all biopsies) and trimmed to the actual count of training and test samples.

In [55]:
def build_dataset_for_biopsies(
    biopsy_list,
    df,
    folder,
    global_prod, global_sum, local_ratio, local_diff,
    use_list, k_list,
    no_plots=False, 
    all_plots=False
):
    """
    Load and process each biopsy in `biopsy_list`, create tissue/NT masks, select wavenumbers,
    and build the feature array X plus label array y for all in-tissue pixels across these biopsies.

    Returns
    -------
    X_full : np.ndarray of shape (N, n_keys)   (all in-tissue pixels from the biopsies)
    y_full : np.ndarray of shape (N,)         (same shape, the labels)
    r_full, c_full : row/column pixel indices of shape (N,) for reference
    b_full : index of which biopsy each pixel came from (shape (N,))
    key_inds : the wavenumber indices used in make_D_arrays
    wavenumbers : the array of wavenumbers from the last loaded biopsy
    """
    n_bio = len(biopsy_list)
    n_rows_max = 512
    n_cols_max = 512

    X_list = []
    y_list = []
    r_list = []
    c_list = []
    b_list = []

    wavenumbers = None
    key_inds    = None

    for b_idx, i_bio in enumerate(biopsy_list):

        # get row_T, col_T, row_NT, col_NT, etc.
        n_biopsies, bio_label, row_T, col_T, row_NT, col_NT, b_type = select_image(i_bio, df)

        # LOAD FTIR data
        FTIR_file, FTIR, wavenumbers_local = get_file_and_cube(bio_label, folder)
        if wavenumbers is None:
            wavenumbers = wavenumbers_local

        n_rows = FTIR.shape[0]
        n_cols = FTIR.shape[1]

        #  any plots or pre-checks
        k_image = 3000
        plot_image(no_plots, k_image, FTIR, wavenumbers_local, bio_label)

        # raw/processed spectrum just for demonstration
        row_spect_raw, col_spect_raw, spectrum_raw = plot_raw_spectrum(
            no_plots, row_T, col_T, row_NT, col_NT, FTIR, wavenumbers_local
        )
        k_min_proc, k_max_proc, k_columns = plot_proc_spectrum(
            no_plots, bio_label, row_spect_raw, col_spect_raw, spectrum_raw, df, wavenumbers_local
        )

        plot_FTIR_NTpix(no_plots, k_image, n_rows, n_cols, FTIR, wavenumbers_local, col_T, row_T, col_NT, row_NT)

        #TISSUE & NT MASKS
        q_thresh = 0.1
        k_low, k_high, in_tissue = make_tissue_mask(no_plots, q_thresh, n_rows, n_cols, FTIR, wavenumbers_local)
        NT_mask = make_NT_mask(no_plots, n_rows, n_cols, row_T, col_T, row_NT, col_NT)


        NT_mask_spect_raw, NT_mask_spect_proc = plot_pixel_NT_spectra(
            no_plots, k_min_proc, k_max_proc, k_columns, FTIR, NT_mask, wavenumbers_local,
            df, bio_label, row_spect_raw, col_spect_raw, col_T, col_NT
        )
        absorbance_hist(no_plots, k_low, k_high, in_tissue, NT_mask, n_rows, n_cols, FTIR, wavenumbers_local)
        in_tissue_spect, in_tissue_NT_spect, in_tissue_notNT_spect = plot_in_out_spectra(
            no_plots, in_tissue, NT_mask, k_min_proc, k_max_proc, n_rows, n_cols, FTIR, wavenumbers_local
        )
        plot_multi_k_image(no_plots, all_plots, in_tissue_spect, NT_mask_spect_raw, NT_mask_spect_proc,
                           k_min_proc, k_max_proc, n_rows, n_cols, FTIR, wavenumbers_local, k_columns)

        if key_inds is None:
            # e.g. use your function select_wavenumbers
            key_inds_local = select_wavenumbers(
                no_plots, NT_mask_spect_raw, NT_mask_spect_proc,
                k_list, 2500, 3700, k_min_proc, k_max_proc,
                wavenumbers_local, in_tissue_spect, k_columns, use_list
            )
            key_inds = key_inds_local
        else:
            key_inds_local = key_inds  # re-use the same for all biopsies

        # build 3D (n_rows, n_cols, n_keys) => X_D, y_D
        X_D_3D, y_D_2D = make_D_arrays(
            key_inds_local, in_tissue, NT_mask,
            global_prod, global_sum, local_ratio, local_diff,
            norm_cut=1e-4,
            n_rows=n_rows, n_cols=n_cols,
            b_type=b_type,
            FTIR=FTIR, wavenumbers=wavenumbers_local
        )
        plot_X_D_arr(all_plots, X_D_3D, y_D_2D, n_rows, n_cols)

        # flatten only the in_tissue pixels
        row_idxs, col_idxs = np.where(in_tissue == True)
        X_flat = X_D_3D[row_idxs, col_idxs, :]
        y_flat = y_D_2D[row_idxs, col_idxs]
        
        # store row/col/biopsy for reference
        r_list.append(row_idxs)
        c_list.append(col_idxs)
        b_list.append(b_idx * np.ones_like(row_idxs))  # or store i_bio directly

        X_list.append(X_flat)
        y_list.append(y_flat)

    # end for each biopsy in biopsy_list

    # aconcatenate
    if len(X_list) > 0:
        X_full = np.concatenate(X_list, axis=0)
        y_full = np.concatenate(y_list, axis=0)
        r_full = np.concatenate(r_list, axis=0)
        c_full = np.concatenate(c_list, axis=0)
        b_full = np.concatenate(b_list, axis=0)
    else:
        # no biopsies => empty
        X_full = np.empty((0, len(key_inds) if key_inds else 0))
        y_full = np.empty((0,))
        r_full = np.empty((0,))
        c_full = np.empty((0,))
        b_full = np.empty((0,))

    return X_full, y_full, r_full, c_full, b_full, wavenumbers, key_inds


## Image training and test data
- add n_rows and n_cols are arguments
- code handles both cases where multiple rows of subplots are created (in which case ax is a 2D array) and when only one row is created (ax is 1D).

In [56]:
def plot_train_test_data(all_plots, X_train, y_train, c_train, r_train,
                         X_test, y_test, c_test, r_test, n_rows, n_cols):
    """
    Plot training and test data as scatter plots for chosen feature (wavenumber) indices
    
    For a selected range of feature indices (here from k_ind_min to k_ind_max with a fixed step),
    the function creates 2 sets of plots:
      - One for training data
      - One for test data.
    
    In each subplot, the feature values (with a viridis colormap) are plotted at the pixel locations 
    (using c_train, r_train or c_test, r_test), with a semi-transparent overlay of the target values 
    (using a Grays colormap). The target overlay helps visualize the segmentation or class labels.
    
    Parameters
    ----------
    all_plots : if True, the plots will be displayed.
    X_train : training feature data.
    y_train : training target data.
    c_train : column indices for training samples.
    r_train : row indices for training samples.
    X_test : test feature data.
    y_test : test target data.
    c_test : column indices for test samples.
    r_test : row indices for test samples.
    n_rows : number of rows in the original image.
    n_cols : number of columns in the original image.
    
    Returns
    -------
    None
    """
    if all_plots:
        print("\nPlot training and test data")
        use_global_v = True  # Use global v_min/v_max for all plots
        
        # Define the range of feature (wavenumber) indices to plot.
        k_ind_min = 0
        k_ind_max = 10
        # step length for plots (indices)
        k_ind_step = 2
        n_plots = (k_ind_max - k_ind_min) // k_ind_step
        n_cols_plot = 4
        n_rows_plot = n_plots // n_cols_plot + (n_plots % n_cols_plot > 0)
        
        #plot Training Data
        print("\nTraining data\n")
        #scaled based on the image dimensions
        fig, ax = plt.subplots(n_rows_plot, n_cols_plot, 
                                figsize=(2.5 * n_cols_plot * n_cols / 128, 2.5 * n_rows_plot * n_rows / 128))
        
        q_min = 0.01
        q_max = 0.99
        if use_global_v:
            # Calc global colour limits over the selected feature indices
            v_min = np.quantile(X_train[:, k_ind_min:k_ind_max], q_min)
            v_max = np.quantile(X_train[:, k_ind_min:k_ind_max], q_max)
        opacity = 0.05
        k_ind = k_ind_min
        
        # case where subplots are in a 2D array
        if n_rows_plot > 1:
            for r in range(n_rows_plot):
                for c in range(n_cols_plot):
                    if k_ind >= k_ind_max:
                        ax[r, c].axis('off')
                        continue
                    if not use_global_v:
                        v_min = np.quantile(X_train[:, k_ind], q_min)
                        v_max = np.quantile(X_train[:, k_ind], q_max)
                    # Plot the feature data.
                    scatter2d(ax[r, c], c_train, r_train, X_train[:, k_ind],
                              title=f"k index = {k_ind}",
                              x_label='Column', y_label='Row', c_map='viridis')
                    # Overlay target data (assumes y_train is 1 or 2D with 1 column)
                    if y_train.ndim == 1:
                        scatter2d(ax[r, c], c_train, r_train, y_train,
                                  title=f"k index = {k_ind}",
                                  x_label='Column', y_label='Row', c_map='Grays', alpha=opacity)
                    else:
                        scatter2d(ax[r, c], c_train, r_train, y_train[:, 0],
                                  title=f"k index = {k_ind}",
                                  x_label='Column', y_label='Row', c_map='Grays', alpha=opacity)
                    k_ind += k_ind_step
        else:
            # only one row of subplots, ax is 1D
            for i in range(n_cols_plot):
                if k_ind >= k_ind_max:
                    ax[i].axis('off')
                    continue
                if not use_global_v:
                    v_min = np.quantile(X_train[:, k_ind], q_min)
                    v_max = np.quantile(X_train[:, k_ind], q_max)
                scatter2d(ax[i], c_train, r_train, X_train[:, k_ind],
                          title=f"k index = {k_ind}",
                          x_label='Column', y_label='Row', c_map='viridis')
                if y_train.ndim == 1:
                    scatter2d(ax[i], c_train, r_train, y_train,
                              title=f"k index = {k_ind}",
                              x_label='Column', y_label='Row', c_map='Grays', alpha=opacity)
                else:
                    scatter2d(ax[i], c_train, r_train, y_train[:, 0],
                              title=f"k index = {k_ind}",
                              x_label='Column', y_label='Row', c_map='Grays', alpha=opacity)
                k_ind += k_ind_step
        plt.tight_layout()
        plt.show()
        
        #plot test data
        print("\nTest data\n")
        fig, ax = plt.subplots(n_rows_plot, n_cols_plot, 
                                figsize=(2.5 * n_cols_plot * n_cols / 128, 2.5 * n_rows_plot * n_rows / 128))
        if use_global_v:
            v_min = np.quantile(X_test[:, k_ind_min:k_ind_max], q_min)
            v_max = np.quantile(X_test[:, k_ind_min:k_ind_max], q_max)
        k_ind = k_ind_min
        if n_rows_plot > 1:
            for r in range(n_rows_plot):
                for c in range(n_cols_plot):
                    if k_ind >= k_ind_max:
                        ax[r, c].axis('off')
                        continue
                    if not use_global_v:
                        v_min = np.quantile(X_test[:, k_ind], q_min)
                        v_max = np.quantile(X_test[:, k_ind], q_max)
                    scatter2d(ax[r, c], c_test, r_test, X_test[:, k_ind],
                              title=f"k index = {k_ind}",
                              x_label='Column', y_label='Row', c_map='viridis')
                    if y_test.ndim == 1:
                        scatter2d(ax[r, c], c_test, r_test, y_test,
                                  title=f"k index = {k_ind}",
                                  x_label='Column', y_label='Row', c_map='Grays', alpha=opacity)
                    else:
                        scatter2d(ax[r, c], c_test, r_test, y_test[:, 0],
                                  title=f"k index = {k_ind}",
                                  x_label='Column', y_label='Row', c_map='Grays', alpha=opacity)
                    k_ind += k_ind_step
        else:
            for i in range(n_cols_plot):
                if k_ind >= k_ind_max:
                    ax[i].axis('off')
                    continue
                if not use_global_v:
                    v_min = np.quantile(X_test[:, k_ind], q_min)
                    v_max = np.quantile(X_test[:, k_ind], q_max)
                scatter2d(ax[i], c_test, r_test, X_test[:, k_ind],
                          title=f"k index = {k_ind}",
                          x_label='Column', y_label='Row', c_map='viridis')
                if y_test.ndim == 1:
                    scatter2d(ax[i], c_test, r_test, y_test,
                              title=f"k index = {k_ind}",
                              x_label='Column', y_label='Row', c_map='Grays', alpha=opacity)
                else:
                    scatter2d(ax[i], c_test, r_test, y_test[:, 0],
                              title=f"k index = {k_ind}",
                              x_label='Column', y_label='Row', c_map='Grays', alpha=opacity)
                k_ind += k_ind_step
        plt.tight_layout()
        plt.show()
    return


## Create new or load existing MLP and train

-  now takes X_train, y_train, bio_list, and bio_mod_list as input parameters
-  If the model file exists, model is loaded and its max_iter and warm_start are updated, so training can continue from the saved state rather than start over.
-  If no model exists, a new MLPClassifier is made.
-  training loss curve is plotted after each iteration to give feedback on the training process.
-  filename is constructed from the hidden layer configuration, the number of keys, and the biopsy lists

## New pytorch cklass

In [57]:
class MyTorchMLP(nn.Module):
    def __init__(
        self, 
        input_dim, 
        hidden_dims=(60, 60, 60, 60), 
        num_classes=3,
        dropout_p=0.7
    ):
        """
        Parameters:
        -----------
        input_dim : int
            Number of input features.
        hidden_dims : tuple of int
            Sizes of each hidden layer.
        num_classes : int
            Number of output classes (size of final layer).
        dropout_p : float
            Probability of dropout (e.g., 0.3 means 30% of neurons randomly dropped).
        """
        super(MyTorchMLP, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        # For each hidden layer in hidden_dims:
        for hdim in hidden_dims:
            # Linear layer
            layers.append(nn.Linear(prev_dim, hdim))
            # ReLU activation
            layers.append(nn.ReLU(inplace=True))  ###########################
            # dropout layer
            layers.append(nn.Dropout(p=dropout_p))
            
            prev_dim = hdim
        
        # Final output layer (no ReLU, typically no dropout after final layer)
        layers.append(nn.Linear(prev_dim, num_classes))
        
        # Put all layers into a Sequential
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        """
        x: shape [batch_size, input_dim]
        Returns logits: shape [batch_size, num_classes]
        """
        return self.network(x)



In [58]:
# def make_and_train_MLP(n_iter, key_inds, file_label, X_train, y_train, bio_list, bio_mod_list, seed=SEED):
#     print("\nMake and train MLP (new model each time)")

#     """
# #     Create new MLP model 
    
# #     Function creates a new MLPClassifier (stopped loading for now as training gives strange behaviour so must affect in some way.
# #     then trains it on the provided training data for a certain number of iterations.
# #     After each iteration, the training performance and loss curve are plotted.
    
# #     Parameters
# #     ----------
# #     n_iter : number of training iterations. If 0, one iteration is performed.
# #     key_inds : indices of the features (wavenumbers) used; its length is used in the model filename.
# #     file_label : label string that becomes part of the model filename.
# #     X_train : training feature data.
# #     y_train :training target labels.
# #     bio_list : list of biopsy identifiers (used in constructing the filename).
# #     bio_mod_list : list of biopsy modifications (used in constructing the filename).
    
# #     Returns
# #     -------
# #     hidden_layers : layer config used in the MLPClassifier.
#     """
    
#     hidden_layers = (60, 60, 60, 60)
#     n_keys = len(key_inds)
    
#     # Build strings for the filename (if you still want to save the final model)
#     h_str = '-' + str(hidden_layers).replace(", ", "-")[1:-1]
#     bio_str = str(bio_list).replace(", ", "-")[1:-1]
#     bio_mod_str = str(bio_mod_list).replace(", ", "-")[1:-1]
    
#     # file_name = "Barney-" + bio_str + file_label + "-SKwgts-" + str(n_keys) + h_str + "-3-Dev.pkl"
#     if seed is not None:
#         file_name = f"Barney-{bio_str}{file_label}-seed{seed}-SKwgts-{n_keys}{h_str}-Dev.pkl"
#     else:
#         file_name = f"Barney-{bio_str}{file_label}-SKwgts-{n_keys}{h_str}-Dev.pkl"
    
#     additional_iter = 1000  # additional iterations per training call
#     all_losses = []  # list for the loss values
    
#     # changed to always use new model - seems to fix some strange loss behaviour?
#     mlp_classifier = MLPClassifier(solver='sgd', alpha=1e-5, 
#                                    hidden_layer_sizes=hidden_layers, 
#                                    max_iter=additional_iter, warm_start=True, random_state=SEED)
    
#     for n in range(max(1, n_iter)):
#         print(f"\nIteration {n}")
#         mlp_classifier.fit(X_train, y_train)
#         # Accumulate the loss curve:
#         all_losses.extend(mlp_classifier.loss_curve_)
#         print(f"Training set score: {mlp_classifier.score(X_train, y_train)}")
#         print(f"Training set loss: {mlp_classifier.loss_}")
#         plt.figure(figsize=(4,3))
#         plt.plot(all_losses, linestyle='-', color='r')
#         plt.title(f"Training Loss")
#         plt.grid(True)
#         plt.tight_layout()                
#         plt.show()
    
#     joblib.dump(mlp_classifier, Path(file_name))
    
#     final_loss = all_losses[-1] if all_losses else None
#     print("Saving MLP model to:", file_name)
#     return hidden_layers, final_loss, file_name


def make_and_train_pytorch_MLP(n_iter, key_inds, file_label,
                               X_train, y_train,
                               bio_list, bio_mod_list,
                               seed=9834, hidden_dims=(60,60,60,60),
                               num_classes=3, dropout_p=0):
    """
    Creates a PyTorch MLP model, trains it, and saves it to disk.
    
    - n_iter: We'll interpret as 'epochs'.
    - key_inds: used to figure out input_dim from X_train
    - X_train, y_train: NumPy arrays
    - bio_list, bio_mod_list: used for naming the saved file
    - seed: random seed
    - hidden_dims: tuple for hidden layer sizes
    - num_classes: number of classes in the final layer.
    """
    import numpy as np
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import torch.nn.functional as F
    from pathlib import Path

    torch.manual_seed(seed)
    np.random.seed(seed)
    

    X_train_t = torch.from_numpy(X_train).float().to(device)  # shape [N, n_features]
    y_train_t = torch.from_numpy(y_train).long().to(device)    # shape [N], for classification

    input_dim = X_train.shape[1]
    

    model = MyTorchMLP(input_dim, hidden_dims=hidden_dims, num_classes=num_classes, dropout_p=dropout_p)
    model.to(device)
    # Define optimizer and loss function
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    
    # We'll do batch training. You can also do full-batch if data is not huge.
    dataset = torch.utils.data.TensorDataset(X_train_t, y_train_t)
    loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
    
    # We'll store losses so we can plot them (just like scikit 'loss_curve_')
    all_losses = []
    
    # Start training
    model.train()
    for epoch in range(n_iter):
        running_loss = 0.0
        for batch_x, batch_y in loader:
            optimizer.zero_grad()
            logits = model(batch_x)       # shape [batch_size, num_classes]
            loss = criterion(logits, batch_y)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * batch_x.size(0)
        
        epoch_loss = running_loss / len(loader.dataset)
        all_losses.append(epoch_loss)
        
        # Print or plot the epoch loss
        print(f"[Epoch {epoch+1}/{n_iter}] Loss: {epoch_loss:.4f}")
    

    h_str = "-" + str(hidden_dims).replace(", ", "-")[1:-1]
    bio_str = str(bio_list).replace(", ", "-")[1:-1]
    bio_mod_str = str(bio_mod_list).replace(", ", "-")[1:-1]
    
    if seed is not None:
        file_name = f"Barney-{bio_str}{file_label}-seed{seed}-PTwgts-{len(key_inds)}{h_str}-Dev.pth"
    else:
        file_name = f"Barney-{bio_str}{file_label}-PTwgts-{len(key_inds)}{h_str}-Dev.pth"
    
    torch.save(model.state_dict(), file_name)
    print(f"Saved PyTorch model state to {file_name}")
    
    return model, all_losses, file_name



In [59]:
def pytorch_predict(model, X_data):
    """
    Equivalent of scikit-learn's 'predict' and 'predict_proba' for your PyTorch model.
    X_data: NumPy array of shape [N, n_features].
    Returns:
      preds_np: an array of shape [N], class predictions (0,1,2,...).
      probs_np: an array of shape [N, num_classes], predicted probabilities.
    """
    # X_t = torch.from_numpy(X_data).float()  # convert to tensor
    
    model.eval()  
    model.to(device)   # <<<  on the device

    X_t = torch.from_numpy(X_data).float().to(device)   # put data on device
    with torch.no_grad():
        logits = model(X_t)          # shape [N, num_classes]
        # apply softmax to get predicted probabilities
        probs = F.softmax(logits, dim=1)  # shape [N, num_classes]
        preds = torch.argmax(probs, dim=1)  # shape [N]
    
    preds_np = preds.cpu().numpy()
    probs_np = probs.cpu().numpy()
    
    return preds_np, probs_np


## Look at MLP results
- key_inds, X_test, r_test, c_test etc as parameters
- filename for the MLP model constructed using hidden layer configuration, no of features (from key_inds)and biopsy lists. Depending on whether bio_mod_list equals bio_list, either standard or alt model file loaded.
- function uses the loaded model to predict labels for X_test

In [60]:
def test_MLP(no_plots, model_file_name, hidden_layers, 
             n_rows_bio, n_cols_bio, b_test, NT_mask_bio, 
             key_inds, X_test, r_test, c_test, n_cols, n_rows):
    """
    Test PyTorch MLP 

    Parameters
    ----------
    no_plots : if True, no plots will be generated.
    model_file_name : the .pth file (PyTorch weights) we saved earlier.
    hidden_layers : hidden layer configuration used in the MLP model.
    n_rows_bio, n_cols_bio, b_test, NT_mask_bio, key_inds, X_test, r_test, c_test, n_cols, n_rows :
        same as before, used for dimension references, 
        plotting, or array indexing.

    Returns
    -------
    X_test_MLP : predicted labels for the test set as returned by the PyTorch model.
    """

    
    print("\nTest MLP (PyTorch version)")

    from your_module import MyTorchMLP  # or wherever you defined it
    input_dim = X_test.shape[1]
    num_classes = 3  # or 2, depending on your classification scheme
    
    model = MyTorchMLP(input_dim=input_dim, hidden_dims=hidden_layers, num_classes=num_classes, dropout_p=dropout_p)
    model.to(device)

    checkpoint = torch.load(Path(model_file_name))
    model.load_state_dict(checkpoint)
    model.eval()  # set dropout/batchnorm 

    X_test_t = torch.from_numpy(X_test).float().to(device)
    

    with torch.no_grad():
        logits = model(X_test_t)          # shape [N, num_classes]
        probs = F.softmax(logits, dim=1)  # shape [N, num_classes]
        preds = torch.argmax(probs, dim=1) # shape [N]
    
    X_test_MLP = preds.cpu().numpy()  # shape [N]
    print("\nX_test_MLP.shape", X_test_MLP.shape)

    import numpy as np
    import matplotlib.pyplot as plt

    unique_test_bios = np.unique(b_test)
    if not no_plots:
        for global_nb in unique_test_bios:
            bool_pix = (b_test == global_nb)
            
            fig = plt.figure(figsize=(5 * n_cols_bio[global_nb] / 128,
                                      2.5 * n_rows_bio[global_nb] / 128))
            axA = fig.add_subplot(1, 2, 1)
            axA.set_title(f"PyTorch Model; test biopsy {global_nb}")
            axA.set_xlabel("Col")
            axA.set_ylabel("Row")
            imA = axA.scatter(c_test[bool_pix], r_test[bool_pix],
                              c=X_test_MLP[bool_pix], vmin=0, vmax=num_classes-1, cmap="plasma")
            axA.set_xlim(0.0, n_cols_bio[global_nb])
            axA.set_ylim(0.0, n_rows_bio[global_nb])
            axA.axis('equal')
            
            axB = fig.add_subplot(1, 2, 2)
            axB.set_title("NT Mask")
            axB.set_xlabel("Col")
            axB.set_ylabel("Row")
            axB.imshow(NT_mask_bio[global_nb], cmap='Greys')
            axB.set_xlim(0.0, n_cols_bio[global_nb])
            axB.set_ylim(0.0, n_rows_bio[global_nb])
            
            fig.colorbar(imA)
            plt.tight_layout()
            plt.show()
    
    return X_test_MLP



## Confusion matrix

Structure for two classes is:

|           |         | Predicted Class |         |
|-----------|---------|-----------------|---------|
|           |         | Positive        |Negative |
|**Actual** |Positive | TP              | FN      |
|**Class**  |Negative | FP              | TN      |

Here, TP = True positive, FN = False negative etc.

In terms of indices (selecting for class 0):

|           |   | Predicted Class |    |
|-----------|---|-----------------|----|
|           |   | 0               | 1  |
|**Actual** | 0 | TP              | FN |
|**Class**  | 1 | FP              | TN |


The total number of positive classes ($P = TP + FN$) is fixed, as is the total number of negative classes ($N = FP + TN$), so the confusion matrix has two degrees of freedom.

Writing as a matrix:

\begin{align}
C = \begin{pmatrix} C_{0,0}\ C_{0,1} \\
                    C_{1,0}\ C_{1,1} \end{pmatrix}
\end{align}

The sensitivity (SN) or true positive rate (TPR) is TP/(TP + FN), while the specificity (SP) or true negative rate (TPR) is TN/(TN + FP), i.e., selecting for class 0:

\begin{align}
\text{TPR} &= \frac{C_{0, 0}}{C_{0, 0} + C_{0, 1}} \\ 
\text{TNR} &= \frac{C_{1, 1}}{C_{1, 0} + C_{1, 1}}.
\end{align}

Extend this to three classes, selecting for class 1:

|           |  | Predicted Class |    |    |
|-----------|--|-----------------|----|----|
|           |  | 0               | 1  | 2  |
|**Actual** |0 | TN              | FP | FN |
|**Class**  |1 | FN              | TP | FN |
|           |2 | FN              | FP | TN |

\begin{align}
C = \begin{pmatrix} C_{0,0}\ C_{0,1}\ C_{0,2} \\
                    C_{1,0}\ C_{1,1}\ C_{1,2} \\
                    C_{2,0}\ C_{2,1}\ C_{2,2} \end{pmatrix}
\end{align}

\begin{align}
\text{TPR} &= \frac{C_{1, 1}}{C_{1,0} + C_{1, 1} + C_{1, 2}} \\
\text{TNR} &= \frac{C_{0, 0} + C_{0, 2} + C_{2, 0} + C_{2, 2}}{C_{0,0} + C_{0, 1} + C_{0, 2} + C_{2, 0} + C_{2, 1} + C_{2, 2}}.
\end{align}

Selecting for class 2:

|           |  | Predicted Class |    |    |
|-----------|--|-----------------|----|----|
|           |  | 0               | 1  | 2  |
|**Actual** |0 | TN              | FN | FP |
|**Class**  |1 | FN              | TN | FP |
|           |2 | FN              | FN | TP |

\begin{align}
\text{TPR} &= \frac{C_{2, 2}}{C_{2, 0} + C_{2, 1} + C_{2, 2}} \\
\text{TNR} &= \frac{C_{0, 0} + C_{0, 1} + C_{1, 0} + C_{1, 1}}{C_{0, 0} + C_{0, 1} + C_{0, 2} + C_{1, 0} + C_{1, 1} + C_{1, 2}}.
\end{align}

- copy of X_test_here is made and thresholded so that values below 0.5 become 0, values between 0.5 and 1.5 become 1, and values 1.5 or above become 2.
- confusion matrix is computed via nested loops over the three classes
- if no_plots is False, the confusion matrix is displayed using ConfusionMatrixDisplay

In [61]:
def calc_confusion(no_plots, X_test_here, y_test_here, b_type):
    """a
    Confusion matrix for three classes based on the predictions in X_test_here.
    
    Predictions in X_test_here are thresholded into three classes:
      - Values < 0.5 assigned to class 0.
      - Values between 0.5 and 1.5 assigned to class 1.
      - Values ≥ 1.5 assigned to class 2.
    
    A 3x3 confusion matrix is calculated:
      - confMat[i, j] is the number of samples with actual class i and predicted class j.
    
    For a given b_type (expected to be either 1 or 2), the function computes:
      - TPR for the selected class:
            TPR = C_{b_type, b_type} / (sum of row b_type)
      - TNR for the selected class:
            For b_type == 1:
                TNR = (C_{0,0} + C_{0,2} + C_{2,0} + C_{2,2}) /
                      (C_{0,0} + C_{0,1} + C_{0,2} + C_{2,0} + C_{2,1} + C_{2,2})
            For b_type == 2:
                TNR = (C_{0,0} + C_{0,1} + C_{1,0} + C_{1,1}) /
                      (C_{0,0} + C_{0,1} + C_{0,2} + C_{1,0} + C_{1,1} + C_{1,2})
    
    "diagonality" is also computed as:
    
        diag = (C_{0,0} + C_{1,1} + C_{2,2} - C_{1,0} - C_{1,2} - C_{2,0} - C_{2,1}) /
               (C_{0,0} + C_{1,1} + C_{2,2} + C_{1,0} + C_{1,2} + C_{2,0} + C_{2,1})
    
    Parameters
    ----------
    no_plots :if True, no plot will be displayed.
    X_test_here : array of prediction values (floats) for the test set. This is modified internally.
    y_test_here : array of true class labels (integers: 0, 1, or 2) for the test set.
    b_type : class for which TPR and TNR are computed (expected to be 1 or 2).
    
    Returns
    -------
    confMat : 3x3 confusion matrix.
    diag : computed diagonality measure.
    TPR : TPR for the selected class.
    TNR : TNR for the selected class.
    """
    print("\nCalculate confusion matrix")
    
    # Threshold predictions into three classes.
    # (Make a copy so as not to modify the input array outside the function.)
    preds = X_test_here.copy()
    preds[preds < 0.5] = 0
    preds[np.logical_and(preds >= 0.5, preds < 1.5)] = 1
    preds[preds >= 1.5] = 2
    
    # Compute the 3x3 confusion matrix
    confMat = np.zeros((3, 3))
    for actual in range(3):
        for pred in range(3):
            confMat[actual, pred] = np.sum((y_test_here == actual) & (preds == pred))
    
    # calc of "diagonality".
    bot = (confMat[0, 0] + confMat[1, 1] + confMat[2, 2] +
           confMat[1, 0] + confMat[1, 2] + confMat[2, 0] + confMat[2, 1])
    diag = (confMat[0, 0] + confMat[1, 1] + confMat[2, 2] -
            confMat[1, 0] - confMat[1, 2] - confMat[2, 0] - confMat[2, 1]) / bot
    
    # Compute TPR TNR based on b_type.
    if b_type == 1:
        TPR = confMat[1, 1] / (confMat[1, 0] + confMat[1, 1] + confMat[1, 2])
        TNR = (confMat[0, 0] + confMat[0, 2] + confMat[2, 0] + confMat[2, 2]) / (
              confMat[0, 0] + confMat[0, 1] + confMat[0, 2] +
              confMat[2, 0] + confMat[2, 1] + confMat[2, 2])
    elif b_type == 2:
        TPR = confMat[2, 2] / (confMat[2, 0] + confMat[2, 1] + confMat[2, 2])
        TNR = (confMat[0, 0] + confMat[0, 1] + confMat[1, 0] + confMat[1, 1]) / (
              confMat[0, 0] + confMat[0, 1] + confMat[0, 2] +
              confMat[1, 0] + confMat[1, 1] + confMat[1, 2])
    else:
        print(f"\nUnexpected b_type in calc_confusion: {b_type}")
        TPR, TNR = np.nan, np.nan
    
    print("\nConfusion matrix:\n", confMat)
    print(f"\nDiagonality: {diag:.3f}")
    print(f"True positive rate (TPR): {TPR:.3f}")
    print(f"True negative rate (TNR): {TNR:.3f}")
    
    if not no_plots:
        print("\nPlot of confusion matrix:")
        disp = ConfusionMatrixDisplay(confusion_matrix=confMat, display_labels=[0, 1, 2])
        disp.plot(cmap=plt.cm.Blues)
        plt.show()
    
    return confMat, diag, TPR, TNR


## K nearest neighbors - optimise k
- accepts X_train and y_train as parameters
- Using np.arange(1, k_max + 1) produces an array of integer k values from 1 up to k_max
-For each k value, function calculates average and standard deviation of the accuracy scores using cross validation

In [62]:
def optimize_k(no_plots, k_max, X_train, y_train, cv=5):
    """
    Optimise the number of neighbors (k) for KNN classification using cross validation.
    
    Function computes the cross-validated accuracy score for KNN classifiers with k values
    ranging from 1 to k_max. Returns the average accuracy scores and the standard deviations
    for each k value.
    
    Parameters
    ----------
    no_plots : if True, optimization plot wont be displayed.
    k_max : maximum number of neighbours to consider (i.e., k values range from 1 to k_max).
    X_train : training feature data.
    y_train : training target labels.
    cv : number of cross-validation folds (default to 5).
    
    Returns
    -------
    score_avg :average accuracy score for each k value.
    score_rms : standard dev of the accuracy scores for each k value.
    """
    print(f"\nOptimise k value for KNN, max k = {k_max}")
    
    # Generate k values from 1 to k_max.
    k_values = np.arange(1, k_max + 1)
    
    score_avg = np.zeros(len(k_values))
    score_rms = np.zeros(len(k_values))
    
    for i, k in enumerate(k_values):
        knn = KneighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, X_train, y_train, cv=cv)
        score_avg[i] = np.mean(scores)
        score_rms[i] = np.std(scores)
    
    if not no_plots:
        fig, ax = plt.subplots(1, 1, figsize=(5, 3))
        ax.errorbar(k_values, score_avg, yerr=score_rms, marker='o', color='r', linestyle='--')
        ax.set_title("K Optimisation")
        ax.set_xlabel("K Values")
        ax.set_ylabel("Accuracy Score")
        ax.grid(color='g')
        plt.tight_layout()
        plt.show()
    
    return score_avg, score_rms


## Train with selected k
- receives key_inds, bio_list, X_train, and y_train as parameters
- biopsy list converted to a string with commas replaced by dashes and square brackets removed. The filename also incl the no of features (from len(key_inds)) and the no of neighbors.
- if model file already exists, function prints a message and doesnt retrain the model. Otherwise trains a new KNN classifier and saves it using joblib.

In [63]:
def make_and_train_KNN(no_plots, n_neighbors, file_label, key_inds, bio_list, X_train, y_train):
    """
    Create and train a KNN classifier with selected number of neighbors,
    or load an existing trained model if available.
    
    Model is saved to disk with a filename incl the training biopsy identifiers,
    a file label, the number of features (derived from key_inds), and the selected number of neighbors.
    
    Parameters
    ----------
    no_plots : control plotting output (not used in this function, but kept for consistency).
    n_neighbors :number of neighbors to use in the KNN classifier.
    file_label : label string to include in model filename.
    key_inds : indices of the features (wavenumbers) used; its length determines the number of features.
    bio_list : list of biopsy identifiers used in training.
    X_train : training feature data.
    y_train : training target labels.
    
    Returns
    -------
    None
    """
    n_keys = len(key_inds)
    
    print("\nMake and train KNN with", n_neighbors, "neighbors")
    
    # make a string 
    bio_str = str(bio_list).replace(', ', '-').replace('[', '').replace(']', '')
    
    # create model filename.
    file_name = f"Barney-{bio_str}{file_label}-KNNmod-{n_keys}-{n_neighbors}-Dev.pkl"
    model_file = Path(file_name)
    
    if model_file.is_file():
        print("KNN training already performed, using file", file_name)
    else:
        print("Creating new KNN model file", file_name)
        #make and  train the KNN classifier.
        knn_classifier = KneighborsClassifier(n_neighbors=n_neighbors)
        knn_classifier.fit(X_train, y_train)
        print("Saving file", file_name)
        joblib.dump(knn_classifier, model_file)
    
    return


## Test KNN
- key_inds, hidden_layers, X_test, r_test, c_test, n_cols, n_rows are passed as parameters
- constructs two possible filenames based on the training biopsy list and alternative list, selects the appropriate file to load using joblib.load.
- loaded KNN classifier is used to predict labels for the test set, stored in X_test_KNN

In [64]:
def test_KNN(no_plots, bio_list, bio_mod_list, n_neighbors, n_rows_bio, n_cols_bio, b_test, NT_mask_bio, file_label, key_inds, hidden_layers, X_test, r_test, c_test, n_cols_max, n_rows_max):
    """
    Test the KNN model on the test set and display plots.
    
    Function loads a saved KNN model file with filename is constructed using the training biopsy list,
    a file label, the number of features (from key_inds), and the selected number of neighbors.
    If the test biopsy list (bio_mod_list) differs from the training biopsy list (bio_list), an alternative model
    file is used. The loaded model is then used to predict labels on the test set. For each biopsy, a figure is
    generated that shows a scatter plot of the predicted labels over the pixel coordinates (using a plasma
    colormap) and the corresponding NT mask image.
    
    Parameters
    ----------
    no_plots : if True, no plots will be generated.
    bio_list : list of biopsy identifiers used during training.
    bio_mod_list : list of biopsy identifiers used for testing.
    n_neighbors : number of neighbors used in the KNN model.
    n_rows_bio : list of the number of rows for each biopsy in the test set.
    n_cols_bio : list of the number of columns for each biopsy in the test set.
    b_test : array of biopsy indices corresponding to each test sample.
    NT_mask_bio : list of NT mask arrays (one per biopsy) for the test set.
    file_label : llabel string that becomes part of the model filename.
    key_inds : indices of the features used; its length is used in the filename.
    hidden_layers :hidden layer configuration used in the KNN model (for filename construction).
    X_test : test feature data.
    r_test : row indices for each test sample.
    c_test : column indices for each test sample.
    n_cols : number of columns (used for figure sizing).
    n_rows : overall number of rows (used for figure sizing).
    
    Returns
    -------
    X_test_KNN : predicted labels for the test set.
    """
    print("\nTest KNN")
    # whether to load an alternative model if test biopsy list differs from training list.
    load_new_model = not (bio_mod_list == bio_list)
    
    n_keys = len(key_inds)
    h_str = '-' + str(hidden_layers).replace(", ", "-")[1:-1]
    bio_str = str(bio_list).replace(", ", "-")[1:-1]
    bio_mod_str = str(bio_mod_list).replace(", ", "-")[1:-1]
    
    model_file = ("Barney-" + bio_str + file_label +
                  "-KNNmod-" + str(n_keys) + "-" + str(n_neighbors) + "-Dev.pkl")
    new_model_file = ("Barney-" + bio_mod_str + file_label +
                      "-KNNmod-" + str(n_keys) + "-" + str(n_neighbors) + "-Dev.pkl")
    
    print("Training dataset biopsies:", str(bio_list)[1:-1])
    if load_new_model:
        print("Alternative KNN model used for test, biopsies:", str(bio_mod_list)[1:-1])
        print("File:", new_model_file)
        knn_classify = joblib.load(Path(new_model_file))
    else:
        print("Same KNN model used for test, biopsies:", str(bio_list)[1:-1])
        print("File:", model_file)
        knn_classify = joblib.load(Path(model_file))
    
    use_global_v = False
    # Predict test labels.
    X_test_KNN = knn_classify.predict(X_test)
    
    # number of biopsies in the test set.
    n_bio_list = len(bio_list)
    
    if not no_plots:
        print("\nTest data\n")
        for nb in range(n_bio_list):
            bool_pix = (b_test == nb)
            # create figure with size scaled by the overall image dimensions
            fig = plt.figure(figsize=(5 * n_cols / 128, 2.5 * n_rows / 128))
            
            axA = fig.add_subplot(1, 2, 1)
            axA.set_title("Model " + str(bio_mod_list)[1:-1] + f"; test {bio_list[nb]}")
            axA.set_xlabel("Col")
            axA.set_ylabel("Row")
            imA = axA.scatter(c_test[bool_pix], r_test[bool_pix], c=X_test_KNN[bool_pix],
                              vmin=0, vmax=2, cmap="plasma")
            # set axis limits based on the dimensions for current biopsy
            axA.set_xlim(0.0, n_cols_bio[nb])
            axA.set_ylim(0.0, n_rows_bio[nb])
            axA.axis('equal')
            
            axB = fig.add_subplot(1, 2, 2)
            axB.set_title("Mask")
            axB.set_xlabel("Col")
            axB.set_ylabel("Row")
            axB.imshow(NT_mask_bio[nb], cmap='Greys')
            axB.set_xlim(0.0, n_cols_bio[nb])
            axB.set_ylim(0.0, n_rows_bio[nb])
            
            fig.colorbar(imA)
            plt.tight_layout()
            plt.show()
    
    return X_test_KNN


## Run analysis for one set of biopsies

In [65]:
folder = '/Users/oliviashuter/MPhys/YEAR 4/MPhys Project/New Analysis/BarneysData/'

In [ ]:
### import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss


all_biopsies = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
N = len(all_biopsies)

loss_matrix = np.full((N, N), np.nan)
loss_matrix_log10 = np.full((N, N), np.nan)

loss_matrix_file = f"loss_matrix_seed{SEED}.npy"
loss_matrix_log_file = f"loss_matrix_log10_seed{SEED}.npy"

results_dict = {}

# skip the big loop if already trained 
if os.path.exists(loss_matrix_file) and os.path.exists(loss_matrix_log_file):
    loss_matrix = np.load(loss_matrix_file)
    loss_matrix_log10 = np.load(loss_matrix_log_file)
    print("Loaded existing matrices. Skip big loop!")
else:
    # Loop over all biopsy pairs (train_bio, test_bio)
    for i_idx, train_bio in enumerate(all_biopsies):
        for j_idx, test_bio in enumerate(all_biopsies):
            # optional: skip if train_bio == test_bio, 
            # if train_bio == test_bio:
            #    continue

            print(f"\n\n====================================================================")
            print(f"training on {train_bio}, testing on {test_bio}")
            print("=========================================================================\n")

            # Flags for how we transform data (unchanged)
            global_prod = True
            global_sum  = False
            local_ratio = False
            local_diff  = False  # only local_diff is True for now

            scale_data = True

            train_bio_list = [train_bio]
            test_bio_list  = [test_bio]

            # Build TRAIN dataset
            X_train, y_train, r_train, c_train, b_train, wavenumbers_tr, key_inds = build_dataset_for_biopsies(
                biopsy_list=train_bio_list,
                df=df,
                folder=folder,
                global_prod=global_prod,
                global_sum=global_sum,
                local_ratio=local_ratio,
                local_diff=local_diff,
                use_list=True,
                k_list=[2300, 2400, 2485, 2500, 2580, 2600, 2678, 2700, 2800, 2900, 3000,
                        3100, 3200, 3300, 3400, 3500],
                no_plots=True,
                all_plots=False
            )
            print("Train set shape:", X_train.shape, y_train.shape)

            # Build TEST dataset
            X_test, y_test, r_test, c_test, b_test, wavenumbers_ts, key_inds_ts = build_dataset_for_biopsies(
                biopsy_list=test_bio_list,
                df=df,
                folder=folder,
                global_prod=global_prod,
                global_sum=global_sum,
                local_ratio=local_ratio,
                local_diff=local_diff,
                use_list=True,
                k_list=[2300, 2400, 2485, 2500, 2580, 2600, 2678, 2700, 2800, 2900, 3000,
                        3100, 3200, 3300, 3400, 3500],
                no_plots=True,
                all_plots=False
            )
            print("Test set shape:", X_test.shape, y_test.shape)

            if scale_data:
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test  = scaler.transform(X_test)

            # file_label to identify the model
            file_label = "-localdiff" + ("-scale" if scale_data else "-noscale")
            n_iter = 100
            hidden_layers = (60,60,60,60)


            #  TRAIN PYTORCH MLP (replaces scikit MLP)
            # make_and_train_pytorch_MLP:
            #  builds a MyTorchMLP model
            #  trains for n_iter epochs
            #  saves the state_dict to disk
            #  returns (model, all_losses, model_file_name)

            model, all_losses, model_file_name = make_and_train_pytorch_MLP(
                n_iter=n_iter,
                key_inds=key_inds,
                file_label=file_label,
                X_train=X_train,
                y_train=y_train,
                bio_list=train_bio_list,
                bio_mod_list=test_bio_list,
                seed=SEED,
                hidden_dims=hidden_layers,
                num_classes=3  
            )

            print("Trained PyTorch model saved as:", model_file_name)

            # TESTING PYTORCH MODEL
            # predictions on train
            y_pred_train, y_proba_train = pytorch_predict(model, X_train)
            train_loss_val = log_loss(y_train, y_proba_train, labels=[0,1,2])

            # Get predictions on TEST
            y_pred_test, y_proba_test = pytorch_predict(model, X_test)
            test_loss_val  = log_loss(y_test,  y_proba_test,  labels=[0,1,2])
            # test_loss_val  = log_loss(y_test, y_proba_test)
            test_acc       = np.mean(y_pred_test == y_test)

            print(f"Train loss: {train_loss_val:.3f}, "
                  f"Test loss: {test_loss_val:.3f}, "
                  f"Test accuracy: {test_acc:.3f}")

            # Store test_loss_val in the NxN matrix
            loss_matrix[i_idx, j_idx] = test_loss_val
            loss_matrix_log10[i_idx, j_idx] = np.log10(test_loss_val)

            # Put results in a dictionary
            results_key = (train_bio, test_bio)
            results_dict[results_key] = {
                "train_loss": train_loss_val,
                "test_loss":  test_loss_val,
                "test_acc":   test_acc,
                "model_file": model_file_name,
                "all_losses": all_losses
            }

    np.save(loss_matrix_file, loss_matrix)
    np.save(loss_matrix_log_file, loss_matrix_log10)

# Plot the result matrix
plt.figure(figsize=(7,5))
im = plt.imshow(loss_matrix, cmap="viridis", origin="upper")
plt.colorbar(im, label="Test loss (local diff norm)")

tick_labels = [str(b) for b in all_biopsies]
plt.xticks(range(N), tick_labels, rotation=90)
plt.yticks(range(N), tick_labels)
plt.xlabel("Test Biopsy")
plt.ylabel("Train Biopsy")
plt.title("MLP: Train i Test j loss matrix")

plt.tight_layout()
plt.show()

# Also plot log10
plt.figure()
plt.imshow(loss_matrix_log10, cmap='viridis', origin='upper')
plt.colorbar(label="log10(Test Loss)")
plt.xticks(range(N), tick_labels, rotation=90)
plt.yticks(range(N), tick_labels)
plt.xlabel("Test Biopsy")
plt.ylabel("Train Biopsy")
plt.title("MLP: Train i-Test j loss matrix (Dropout = 0.5)")
plt.tight_layout()
plt.show()

print("Final loss matrix shape:", loss_matrix.shape)
print("Final loss_matrix:\n", loss_matrix)

joblib.dump(results_dict, "full16x16resultsseed9834.pkl")
print("\nAll results stored in 'full16x16resultsseed9834.pkl'.\n")

print("Test loss for each:")
for i_idx, train_bio in enumerate(all_biopsies):
    for j_idx, test_bio in enumerate(all_biopsies):
        if i_idx == j_idx:
            continue
        val = loss_matrix[i_idx, j_idx]
        if not np.isnan(val):
            print(f"  Train biopsy {train_bio}, Test biopsy {test_bio} --> Test loss = {val:.3f}")

print("\nDetailed results:\n")
for key, metrics in results_dict.items():
    print(f"Train={key[0]}, Test={key[1]} => "
          f"Train Loss={metrics['train_loss']:.3f}, "
          f"Test Loss={metrics['test_loss']:.3f}, "
          f"Test Acc={metrics['test_acc']:.3f}, "
          f"Model={metrics['model_file']}")

end_time = datetime.datetime.now()
print(f"Script finished at: {end_time}")
print(f"Total execution time: {end_time - start_time}")





training on 0, testing on 0


Select biopsy for study
Number of biopsies available: 17
Chosen biopsy index: 0, label: 12089
Non-transforming area: 0 pixels.
Transforming area: 979 pixels.
Biopsy type: 2


Get FTIR data
Looking at biopsy with label: 12089
Shape of FTIR image: (256, 128, 1506)
Number of wavenumbers: 1506

Plot raw spectrum: choose pixel
Min value of col_T: 79, max value: 121
Min value of row_T: 52, max value: 137
Plot raw spectrum in pixel, row 105 column 104

Plot processed spectrum in particular pixel, row 105 column 104

Plot FTIR image with N/T pixels marked

Make tissue mask
Quantile for threshold 0.10, absorbance for threshold -1.32e-03

Make N/T mask

Plot spectra from N/T pixels

Plot absorbance distribution at selected wavenumbers in/out of tissue, and N/T regions
Minimum absorbance -6.75e-01
Maximum absorbance 8.00e+00

Plot spectra in/out of tissue and N/T regions
Number of pixels in in_tissue spectrum: 29491
Number of pixels in in_tissue_NT spectrum: 979
Nu

## 